In [1]:
#Dumb charging
#!/usr/bin/python
import csv
import numpy as np
from scipy import interpolate
from gurobipy import *

eff = 15
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []

def parameterdc( M, T, Eint,Ereq,TS,TD):
    EI =[0 for t in range(len(Eint))]
    ER =[0 for t in range(len(Ereq))]
    for i in range(0,len(Eint)):
        EI[i] = Eint[i]
        ER[i] = Ereq[i]
    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
        #print("StartTime:" , TS[i])
        #print("DepartureTime:" , TD[i])
        #print("Initial energy:" , Eint[i])
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/household8_profile.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHP_a = float(row[0])
        HHP_b = float(row[1])
        HHP_c = float(row[2])
        HHQ_a = float(row[3])
        HHQ_b = float(row[4])
        HHQ_c = float(row[5])
        HH.append([HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PV_P = float(row[0])
        PV.append(PV_P)
    m = Model("Generalizeparam")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    
    #Phase constraint:
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
    
    #power outside TS TD as 0:
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
            
    #Subcost calculation:
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            e[i][t] +=  p[i,t] * eff * 1/4  # Energy consumed by EV1   1/4 *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]    
            te[i] += e[i][t]
            n = te[i]
        SOC = [0,50,100]    #SOC level in %
        Energy = [0,20,40]  #Energy in kwh
        f = interpolate.interp1d(SOC,Energy)
        x = EI[i] 
        y = ER[i] 
        I_E = f(x)
        R_E = f(y)
        I_E = int(np.round(I_E))
        R_E = int(np.round(R_E))
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
        
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    
    #set objective:
    m.ModelSense = GRB.MAXIMIZE
    m.setObjective(C)
    
    #writing the model:
    m.write('dumbcharging.lp')
    
    # Compute optimal solution:
    m.optimize()
    m.write('dumbcharge.sol')
    #print solution: 

    for v in m.getVars():
        print(v.varName, v.x)
    
    print('Obj:', m.objVal)
    
    with open('dumbcharge.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dumbcharge.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
    return;

Eint = [30,20];
Ereq = [80,80];
TS = [0, 5];       
TD = [12, 17];
parameterdc(2,96,Eint,Ereq,TS,TD)
    

# do IIS
#    print('The model is infeasible; computing IIS')
#    m.computeIIS()
#    if m.IISMinimal:
#      print('IIS is minimal\n')
#    else:
#      print('IIS is not minimal\n')
#    print('\nThe following constraint(s) cannot be satisfied:')
#    for c in m.getConstrs():
#        if c.IISConstr:
#             print('%s' % c.constrName)

Academic license - for non-commercial use only
Optimize a model with 362 rows, 768 columns and 768 nonzeros
Variable types: 192 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 12.5693988
Presolve removed 362 rows and 768 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 16 available processors)

Solution count 2: 13.1381 12.5694 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.313814740000e+01, best bound 1.313814740000e+01, gap 0.0000%
a[0,0] 1.0
a[0,1] 1.0
a[0,2] 1.0
a[0,3] 1.0
a[0,4] 1.0
a[0,5] 1.0
a[0,6] 1.0
a[0,7] 1.0
a[0,8] 1.0
a[0,9] 1.0
a[0,10] 1.0
a[0,11] 1.0
a[0,12] 1.0
a[0,13] 1.0
a[0,14] 1.0
a[0,15] 1.0
a[0,16] 1.0
a[0,17] 1.0
a[0,18] 1.0
a[0,19] 1.0
a[0,20] 1.0
a[0,21] 1.0
a[0,22] 

Power[1,52] 0.0
Power[1,53] 0.0
Power[1,54] 0.0
Power[1,55] 0.0
Power[1,56] 0.0
Power[1,57] 0.0
Power[1,58] 0.0
Power[1,59] 0.0
Power[1,60] 0.0
Power[1,61] 0.0
Power[1,62] 0.0
Power[1,63] 0.0
Power[1,64] 0.0
Power[1,65] 0.0
Power[1,66] 0.0
Power[1,67] 0.0
Power[1,68] 0.0
Power[1,69] 0.0
Power[1,70] 0.0
Power[1,71] 0.0
Power[1,72] 0.0
Power[1,73] 0.0
Power[1,74] 0.0
Power[1,75] 0.0
Power[1,76] 0.0
Power[1,77] 0.0
Power[1,78] 0.0
Power[1,79] 0.0
Power[1,80] 0.0
Power[1,81] 0.0
Power[1,82] 0.0
Power[1,83] 0.0
Power[1,84] 0.0
Power[1,85] 0.0
Power[1,86] 0.0
Power[1,87] 0.0
Power[1,88] 0.0
Power[1,89] 0.0
Power[1,90] 0.0
Power[1,91] 0.0
Power[1,92] 0.0
Power[1,93] 0.0
Power[1,94] 0.0
Power[1,95] 0.0
Obj: 13.138147400000012


In [ ]:
#dayahead scheduling
#!/usr/bin/python
import csv
import numpy as np
from datetime import time,datetime,date
from scipy import interpolate
from gurobipy import *

eff = 15
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []
thisdict = {
  '01.01.2014 00:00:00': 0,
  '01.01.2014 00:15:00': 1,
  '01.01.2014 00:30:00': 2
}
thisdict.update([ ('01.01.2014 00:45:00', 3) , ('01.01.2014 01:00:00', 4) , ('01.01.2014 01:15:00', 5) ,
                 ('01.01.2014 01:30:00', 6),('01.01.2014 01:45:00', 7),('01.01.2014 02:00:00', 8),
                 ('01.01.2014 02:15:00', 9),('01.01.2014 02:30:00', 10),('01.01.2014 02:45:00', 11),
                 ('01.01.2014 03:00:00', 12),('01.01.2014 03:15:00', 13),('01.01.2014 03:30:00', 14) , 
                 ('01.01.2014 03:45:00', 15) , ('01.01.2014 04:00:00', 16) , ('01.01.2014 04:15:00', 17),
                 ('01.01.2014 04:30:00', 18),('01.01.2014 04:45:00', 19),('01.01.2014 05:00:00', 20),
                 ('01.01.2014 05:15:00', 21),('01.01.2014 05:30:00', 22),('01.01.2014 05:45:00', 23),
                 ('01.01.2014 06:00:00', 24),('01.01.2014 06:15:00', 25) , ('01.01.2014 06:30:00', 26),
                 ('01.01.2014 06:45:00', 27),('01.01.2014 07:00:00', 28),('01.01.2014 07:15:00', 29),
                 ('01.01.2014 07:30:00', 30),('01.01.2014 07:45:00', 31),('01.01.2014 08:00:00', 32),
                 ('01.01.2014 08:15:00', 33),('01.01.2014 08:30:00', 34) , ('01.01.2014 08:45:00', 35) , 
                 ('01.01.2014 09:00:00', 36) , ('01.01.2014 09:15:00', 37),('01.01.2014 09:30:00', 38),
                 ('01.01.2014 09:45:00', 39),('01.01.2014 10:00:00', 40),('01.01.2014 10:15:00', 41),
                 ('01.01.2014 10:30:00', 42),('01.01.2014 10:45:00', 43),('01.01.2014 11:00:00', 44),
                 ('01.01.2014 11:15:00', 45) , ('01.01.2014 11:30:00', 46),('01.01.2014 11:45:00', 47),
                 ('01.01.2014 12:00:00', 48),('01.01.2014 12:15:00', 49),('01.01.2014 12:30:00', 50),
                 ('01.01.2014 12:45:00', 51),('01.01.2014 13:00:00', 52),('01.01.2014 13:15:00', 53),
                 ('01.01.2014 13:30:00', 54) , ('01.01.2014 13:45:00', 55) , ('01.01.2014 14:00:00', 56) , 
                 ('01.01.2014 14:15:00', 57),('01.01.2014 14:30:00', 58),('01.01.2014 14:45:00', 59),
                 ('01.01.2014 15:00:00', 60),('01.01.2014 15:15:00', 61),('01.01.2014 15:30:00', 62),
                 ('01.01.2014 15:45:00', 63),('01.01.2014 16:00:00', 64),('01.01.2014 16:15:00', 65) , 
                 ('01.01.2014 16:30:00', 66),('01.01.2014 16:45:00', 67),('01.01.2014 17:00:00', 68),
                 ('01.01.2014 17:15:00', 69),('01.01.2014 17:30:00', 70),('01.01.2014 17:45:00', 71),
                 ('01.01.2014 18:00:00', 72),('01.01.2014 18:15:00', 73),('01.01.2014 18:30:00', 74) , 
                 ('01.01.2014 18:45:00', 75) , ('01.01.2014 19:00:00', 76) , ('01.01.2014 19:15:00', 77),
                 ('01.01.2014 19:30:00', 78),('01.01.2014 19:45:00', 79),('01.01.2014 20:00:00', 80),
                 ('01.01.2014 20:15:00', 81),('01.01.2014 20:30:00', 82),('01.01.2014 20:45:00', 83),
                 ('01.01.2014 21:00:00',84),('01.01.2014 21:15:00', 85) , ('01.01.2014 21:30:00', 86),
                 ('01.01.2014 21:45:00', 87),('01.01.2014 22:00:00', 88),('01.01.2014 22:15:00', 89),
                 ('01.01.2014 22:30:00', 90),('01.01.2014 00:45:00', 91),('01.01.2014 23:00:00', 92),
                 ('01.01.2014 23:15:00', 93),('01.01.2014 23:30:00', 94) , ('01.01.2014 23:45:00', 95)])
def parameterda( M, T, Eint,Ereq,TS,TD):

    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/HH_profile_Scheduling.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHDate = row[0]
        HHP_a = float(row[1])
        HHP_b = float(row[2])
        HHP_c = float(row[3])
        HHQ_a = float(row[4])
        HHQ_b = float(row[5])
        HHQ_c = float(row[6])
        HH.append([HHDate,HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile_Scheduling.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PVDate = row[0]
        PV_P = float(row[1])
        PV.append([PVDate,PV_P])
    m = Model("dayahead")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    #Phase constraint
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
            
    #SOC user constraint
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            e[i][t] +=  p[i,t] * eff * 1/4  # Energy consumed by EV1   1/4 *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]
            te[i] += e[i][t]
            n = te[i]
        initial = IE[i] 
        required = RE 
        I_E = int(np.round(initial))
        R_E = int(np.round(required))
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
    
    #power outside TS TD as 0
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))

    #Grid constraint
    for t in range(T):
        #if HH[i][0] == atime:
        for i in range(M):
    #if i < M :
            pa[i][t] = a[i,t] * p[i,t]
            pb[i][t] = b[i,t] * p[i,t]
            pc[i][t] = c[i,t] * p[i,t]
            
            patot[t] += pa[i][t]
            pbtot[t] += pb[i][t]
            pctot[t] += pc[i][t]
        #print(pa[i][t])
        pahhpv[t] = patot[t] + HH[t][1] - PV[t][1]
        pbhhpv[t] = pbtot[t] + HH[t][2]
        pchhpv[t] = pctot[t] + HH[t][3]
   
    
        pab[t] = pahhpv[t] - pbhhpv[t]
        pbc[t] = pbhhpv[t] - pchhpv[t]
        pca[t] = pchhpv[t] - pahhpv[t]
    
        m.addConstr(pab[t] <= 4.60, name="Power_A_B_maxconstraint")
        m.addConstr(pab[t] >= -4.60, name="Power_A_B_minconstraint")
        m.addConstr(pbc[t] <= 4.60, name="Power_B_C_maxconstraint")
        m.addConstr(pbc[t] >= -4.60, name="Power_B_C_minconstraint")
        m.addConstr(pca[t] <= 4.60, name="Power_C_A_maxconstraint")
        m.addConstr(pca[t] >= -4.60, name="Power_C_A_minconstraint")
    
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    Im_cost = C
    ST = STime[0]
    i = M - 1
    DT = DTime[i]
    for t in range(ST,DT):
        if pab[t] <= 2.3:
            Im_cost += 0.0
        elif pab[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pbc[t] <= 2.3:
            Im_cost += 0.0
        elif pbc[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pca[t] <= 2.3:
            Im_cost += 0.0
        elif pca[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
    
    #set objective:
    m.ModelSense = GRB.MINIMIZE
    Im_C = Im_cost
    #m.setObjective(C)
    m.setObjective(Im_C)

    m.write('dayahead.lp')
    #m.write('testcost.lp')

    # Compute optimal solution
    m.optimize()
    m.write('dayahead.sol')
    #print solution 

    for v in m.getVars():
        print(v.varName, v.x)
    
    print('Obj:', m.objVal)
    
    with open('dayahead.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dayahead.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
# do IIS
#    print('The model is infeasible; computing IIS')
#    m.computeIIS()
#    if m.IISMinimal:
#      print('IIS is minimal\n')
#    else:
#      print('IIS is not minimal\n')
#    print('\nThe following constraint(s) cannot be satisfied:')
#    for c in m.getConstrs():
#        if c.IISConstr:
#             print('%s' % c.constrName)
    return;
def ev1arrdeptime():
    with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EV1sampledata.csv') as f:
        reader = csv.reader(f)
        header = next(reader)
        y = []
        for row in reader :
            ydist = float(row[0])
            yaDate = row[1]
            ydDate = row[2]
            y.append([ydist,yaDate,ydDate])
        ev1 =(random.choice(y))
    return(ev1);
ev_a=[0 for i in range(M)]
ev_d=[0 for i in range(M)]
Ev = [0 for i in range(M)]
IE = [0 for i in range(M)]
RE = 17.0
Ev[0] = ev1arrdeptime()
Ev[1] = ev1arrdeptime()
Date = '01.01.2014 '
Time = ':00'
Starttime =[0 for i in range(M)]
Departtime =[0 for i in range(M)]
for i in range(0,M):
    Starttime[i] = Date + Ev[i][1] +Time
    Departtime[i] = Date + Ev[i][2] +Time
    print('energy:',Ev[i][0])
    IE[i] = RE - Ev[i][0]
    print('Initial energy:',IE[i])
    print('ev_a:',Starttime[i])
    print('ev_d:',Departtime[i])
    for item,val in thisdict.items():
        if item == Starttime[i]:
            ev_a[i] = val
        elif item == Departtime[i]:
            ev_d[i] = val
if ev_a[0] < ev_a[1]:        
    TS = [ev_a[0],ev_a[1]]  
    TD= [ev_d[0],ev_d[1]] 
    print('ev1 arrives before')
    print(TS)
    print(TD)
else:
    TS = [ev_a[1],ev_a[0]]   
    TD= [ev_d[1],ev_d[0]]
    print('ev2 arrives before')
    print(TS)
    print(TD)
#Eint = [30,20];
#Ereq = [80,80];

#EV1_a = datetime(2014, 1, 1, 16, 30)
#atime1 = EV1_a.strftime("%d.%m.%Y %H:%M:%S")
#EV2_a = datetime(2014, 1, 1, 18, 15)
#atime2 = EV2_a.strftime("%d.%m.%Y %H:%M:%S")
#EV1_d = datetime(2014, 1, 1, 19, 15)
#dtime1 = EV1_d.strftime("%d.%m.%Y %H:%M:%S")
#EV2_d = datetime(2014, 1, 1, 21, 45)
#dtime2 = EV2_d.strftime("%d.%m.%Y %H:%M:%S")
#for item,val in thisdict.items():
#    if item == atime1:
#        ev1_a = val
#    elif item == atime2:
#        ev2_a = val
#    elif item == dtime1:
#        ev1_d = val    
#    elif item == dtime2:
#        ev2_d = val        
#TS = [ev1_a,ev2_a]
#TD = [ev1_d,ev2_d]
parameterda(2,96,IE,RE,TS,TD)

In [156]:
#code for random value from csv file
import random
import csv
import time
from threading import Timer
TS = []
TD = []
M = 2
def ev1arrdeptime():
    with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/Arr_Dep_EV.csv') as f:
        reader = csv.reader(f)
        header = next(reader)
        y = []
        for row in reader :
            ydist = float(row[0])
            yaDate = row[1]
            ydDate = row[2]
            y.append([ydist,yaDate,ydDate])
        ev1 =(random.choice(y))        
    return ev1;
def ev2arrdeptime():
    with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/Arr_Dep_EV.csv') as file:
        reader = csv.reader(file)
        header = next(reader)
        x = []
        for row in reader :
            xdist = float(row[0])
            xaDate = row[1]
            xdDate = row[2]
            x.append([xdist,xaDate,xdDate])
        ev2 =(random.choice(x)) 
    return(ev2);
#for i in range(0,M):
#    if i == 0:
#        Ev1 = ev1arrdeptime()
        #print('ev1:',Ev1)
        #print('ev1_a:',Ev1[1])
        #print('ev1_d:',Ev1[2])
#    elif i ==1:
#        time.sleep(30)
#        Ev2 = ev2arrdeptime()
        #print('ev2:',Ev2)
        #print('ev2_a:',Ev2[1])
        #print('ev2_d:',Ev2[2])
for i in range(0,M):

    Ev = [0 for i in range(M)]
    Ev[i] = ev1arrdeptime()
    print('ev :',Ev[i])        
    Date = '01.01.2014 '
    Time = ':00'
    Starttime =[0 for i in range(M)]
    Departtime =[0 for i in range(M)]
    Starttime[i] = Date + Ev[i][1] +Time
    Departtime[i] = Date + Ev[i][2] +Time
        #atime1 = Starttime[i]
        #dtime1 = Departtime[i]
    print('ev1_a:',Starttime[i])
    print('ev1_d:',Departtime[i])
    for item,val in thisdict.items():
            #print(item)
        if item == Starttime[i]:
            ev1_a = val
        elif item == Departtime[i]:
            ev1_d = val
    TS.append(ev1_a) #= [ev1_a]
    TD.append(ev1_d) #TD = [ev1_d]
    print(TS)
    print(TD)
    time.sleep(20)

ev : [5.4, '21:15', '04:15']
ev1_a: 01.01.2014 21:15:00
ev1_d: 01.01.2014 04:15:00
[85]
[17]
ev : [8.4, '13:15', '14:30']
ev1_a: 01.01.2014 13:15:00
ev1_d: 01.01.2014 14:30:00
[85, 53]
[17, 58]


In [ ]:
#!/usr/bin/python
import csv
import numpy as np
from scipy import interpolate
from gurobipy import *
import random
import time
import sys
eff = 15
M=2
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []
thisdict = {
  '01.01.2014 00:00:00': 0,
  '01.01.2014 00:15:00': 1,
  '01.01.2014 00:30:00': 2
}
thisdict.update([ ('01.01.2014 00:45:00', 3) , ('01.01.2014 01:00:00', 4) , ('01.01.2014 01:15:00', 5) ,
                 ('01.01.2014 01:30:00', 6),('01.01.2014 01:45:00', 7),('01.01.2014 02:00:00', 8),
                 ('01.01.2014 02:15:00', 9),('01.01.2014 02:30:00', 10),('01.01.2014 02:45:00', 11),
                 ('01.01.2014 03:00:00', 12),('01.01.2014 03:15:00', 13),('01.01.2014 03:30:00', 14) , 
                 ('01.01.2014 03:45:00', 15) , ('01.01.2014 04:00:00', 16) , ('01.01.2014 04:15:00', 17),
                 ('01.01.2014 04:30:00', 18),('01.01.2014 04:45:00', 19),('01.01.2014 05:00:00', 20),
                 ('01.01.2014 05:15:00', 21),('01.01.2014 05:30:00', 22),('01.01.2014 05:45:00', 23),
                 ('01.01.2014 06:00:00', 24),('01.01.2014 06:15:00', 25) , ('01.01.2014 06:30:00', 26),
                 ('01.01.2014 06:45:00', 27),('01.01.2014 07:00:00', 28),('01.01.2014 07:15:00', 29),
                 ('01.01.2014 07:30:00', 30),('01.01.2014 07:45:00', 31),('01.01.2014 08:00:00', 32),
                 ('01.01.2014 08:15:00', 33),('01.01.2014 08:30:00', 34) , ('01.01.2014 08:45:00', 35) , 
                 ('01.01.2014 09:00:00', 36) , ('01.01.2014 09:15:00', 37),('01.01.2014 09:30:00', 38),
                 ('01.01.2014 09:45:00', 39),('01.01.2014 10:00:00', 40),('01.01.2014 10:15:00', 41),
                 ('01.01.2014 10:30:00', 42),('01.01.2014 10:45:00', 43),('01.01.2014 11:00:00', 44),
                 ('01.01.2014 11:15:00', 45) , ('01.01.2014 11:30:00', 46),('01.01.2014 11:45:00', 47),
                 ('01.01.2014 12:00:00', 48),('01.01.2014 12:15:00', 49),('01.01.2014 12:30:00', 50),
                 ('01.01.2014 12:45:00', 51),('01.01.2014 13:00:00', 52),('01.01.2014 13:15:00', 53),
                 ('01.01.2014 13:30:00', 54) , ('01.01.2014 13:45:00', 55) , ('01.01.2014 14:00:00', 56) , 
                 ('01.01.2014 14:15:00', 57),('01.01.2014 14:30:00', 58),('01.01.2014 14:45:00', 59),
                 ('01.01.2014 15:00:00', 60),('01.01.2014 15:15:00', 61),('01.01.2014 15:30:00', 62),
                 ('01.01.2014 15:45:00', 63),('01.01.2014 16:00:00', 64),('01.01.2014 16:15:00', 65) , 
                 ('01.01.2014 16:30:00', 66),('01.01.2014 16:45:00', 67),('01.01.2014 17:00:00', 68),
                 ('01.01.2014 17:15:00', 69),('01.01.2014 17:30:00', 70),('01.01.2014 17:45:00', 71),
                 ('01.01.2014 18:00:00', 72),('01.01.2014 18:15:00', 73),('01.01.2014 18:30:00', 74) , 
                 ('01.01.2014 18:45:00', 75) , ('01.01.2014 19:00:00', 76) , ('01.01.2014 19:15:00', 77),
                 ('01.01.2014 19:30:00', 78),('01.01.2014 19:45:00', 79),('01.01.2014 20:00:00', 80),
                 ('01.01.2014 20:15:00', 81),('01.01.2014 20:30:00', 82),('01.01.2014 20:45:00', 83),
                 ('01.01.2014 21:00:00',84),('01.01.2014 21:15:00', 85) , ('01.01.2014 21:30:00', 86),
                 ('01.01.2014 21:45:00', 87),('01.01.2014 22:00:00', 88),('01.01.2014 22:15:00', 89),
                 ('01.01.2014 22:30:00', 90),('01.01.2014 00:45:00', 91),('01.01.2014 23:00:00', 92),
                 ('01.01.2014 23:15:00', 93),('01.01.2014 23:30:00', 94) , ('01.01.2014 23:45:00', 95)])
def ev1arrdeptime():
    with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EV1sampledata.csv') as f:
        reader = csv.reader(f)
        header = next(reader)
        y = []
        for row in reader :
            ydist = float(row[0])
            yaDate = row[1]
            ydDate = row[2]
            y.append([ydist,yaDate,ydDate])
        ev1 =(random.choice(y))
    return(ev1);
def parameterdy( i, T, IE,RE,TS,TD):
    M= i
    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/household8_profile.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHP_a = float(row[0])
        HHP_b = float(row[1])
        HHP_c = float(row[2])
        HHQ_a = float(row[3])
        HHQ_b = float(row[4])
        HHQ_c = float(row[5])
        HH.append([HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PV_P = float(row[0])
        PV.append(PV_P)
    m = Model("dynamicscheduling")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    #Phase constraint
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
            
    #SOC user constraint
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            e[i][t] +=  p[i,t] * eff * 1/4  # Energy consumed by EV1   1/4 *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]
            te[i] += e[i][t]
            n = te[i]
        initial = IE[i] 
        required = RE 
        I_E = int(np.round(initial))
        R_E = int(np.round(required))
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
    
    #power outside TS TD as 0
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))

    #Grid constraint
    for t in range(T):
        for i in range(M):
    #if i < M :
            pa[i][t] = a[i,t] * p[i,t]
            pb[i][t] = b[i,t] * p[i,t]
            pc[i][t] = c[i,t] * p[i,t]
            
            patot[t] += pa[i][t]
            pbtot[t] += pb[i][t]
            pctot[t] += pc[i][t]
        #print(pa[i][t])
        pahhpv[t] = patot[t] + HH[t][0] - PV[t]
        pbhhpv[t] = pbtot[t] + HH[t][1]
        pchhpv[t] = pctot[t] + HH[t][2]
   
        
        #pab[t] = patot[t] - pbtot[t]
        #pbc[t] = pbtot[t] - pctot[t]
        #pca[t] = pctot[t] - patot[t]
    
        pab[t] = pahhpv[t] - pbhhpv[t]
        pbc[t] = pbhhpv[t] - pchhpv[t]
        pca[t] = pchhpv[t] - pahhpv[t]
    
        m.addConstr(pab[t] <= 4.60, name="Power_A_B_maxconstraint")
        m.addConstr(pab[t] >= -4.60, name="Power_A_B_minconstraint")
        m.addConstr(pbc[t] <= 4.60, name="Power_B_C_maxconstraint")
        m.addConstr(pbc[t] >= -4.60, name="Power_B_C_minconstraint")
        m.addConstr(pca[t] <= 4.60, name="Power_C_A_maxconstraint")
        m.addConstr(pca[t] >= -4.60, name="Power_C_A_minconstraint")
    
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    Im_cost = C
    ST = STime[0]
    i = M - 1
    DT = DTime[i]
    for t in range(ST,DT):
        if pab[t] <= 2.3:
            Im_cost += 0.0
        elif pab[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pbc[t] <= 2.3:
            Im_cost += 0.0
        elif pbc[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pca[t] <= 2.3:
            Im_cost += 0.0
        elif pca[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
    #set objective:
    m.ModelSense = GRB.MINIMIZE
    Im_C = Im_cost
    #m.setObjective(C)
    m.setObjective(Im_C)

    m.write('Dynamicscheduling.lp')
    #m.write('testcost.lp')

    # Compute optimal solution
    m.optimize()
    m.write('Dynamicscheduling.sol')
    #print solution 

    for v in m.getVars():
        print(v.varName, v.x)
    
    print('Obj:', m.objVal)
    finalcost = m.objVal
         
       # elif i ==1:
       #     time.sleep(30)
       #     Ev2 = ev2adt() #ev2arrdeptime()
       #     print('ev2:',Ev2)
       #     print('ev2_a:',Ev2[1])
       #     print('ev2_d:',Ev2[2])
    return(finalcost);
M = 2
ev_a=[0 for i in range(M)]
ev_d=[0 for i in range(M)]
Ev = [0 for i in range(M)]
IE = [0 for i in range(M)]
RE = 17.0
Ev[0] = ev1arrdeptime()
Ev[1] = ev1arrdeptime()
Date = '01.01.2014 '
Time = ':00'
Starttime =[0 for i in range(M)]
Departtime =[0 for i in range(M)]
for i in range(0,M):
    Starttime[i] = Date + Ev[i][1] +Time
    Departtime[i] = Date + Ev[i][2] +Time
    print('energy:',Ev[i][0])
    IE[i] = RE - Ev[i][0]
    print('Initial energy:',IE[i])
    print('ev_a:',Starttime[i])
    print('ev_d:',Departtime[i])
    for item,val in thisdict.items():
        if item == Starttime[i]:
            ev_a[i] = val
        elif item == Departtime[i]:
            ev_d[i] = val
if ev_a[0] < ev_a[1]:        
    TS = [ev_a[0],ev_a[1]]  
    TD= [ev_d[0],ev_d[1]] 
    print('ev1 arrives before')
    print(TS)
    print(TD)
else:
    TS = [ev_a[1],ev_a[0]]   
    TD= [ev_d[1],ev_d[0]]
    print('ev2 arrives before')
    print(TS)
    print(TD)
if TS[1] < TD[0]:
    print('inbetween')
    Start = [TS[0]]
    Depart = [TS[1]]
    cost1=parameterdy(1,96,IE,RE,Start,Depart)
    with open('Dynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('Dynamicscheduling.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
    Start = [TS[1],TS[1]]
    Depart = [TD[0],TD[1]]
    IE[0]= RE
    cost2=parameterdy(2,96,IE,RE,Start,Depart)
    with open('Dynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('Dynamicscheduling.csv', 'a', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
    finalcost = cost1 + cost2
    print('final optimal cost:',finalcost)
else:
    exit


In [ ]:
#Distance to energy conversion
#Dist = [48.7, 46] # Initial Energy in EV
distance =[]
with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EV2sampledata.csv') as file:
    reader = csv.reader(file)
    header = next(reader)
    x = []
    for row in reader :
        xdist = float(row[0])
        xaDate = row[1]
        xdDate = row[2]
        x.append([xdist,xaDate,xdDate])
    #ev2 =(random.choice(x)) 
    for i in range(len(x)):
        #print(x[i][0])
        distance.append(x[i][0])
print(distance)
Distance = [0,100]    #Distance given /100 km
Energy = [0,17]  #Energy in kwh
f = interpolate.interp1d(Distance,Energy)
for i in range(len(distance)):
    evdistance = distance[i] 
    Energyeq = f(evdistance)
    print('Energy equivalent:',Energyeq)


In [49]:
import csv
def ev1arrdeptime():
    with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EV1sampledata.csv') as f:
        reader = csv.reader(f)
        header = next(reader)
        y = []
        for row in reader :
            yegy = float(row[0])
            yaDate = row[1]
            ydDate = row[2]
            y.append([yegy,yaDate,ydDate])
        ev1 =(random.choice(y))
    return(ev1);
M = 2
ev_a=[0 for i in range(M)]
ev_d=[0 for i in range(M)]
Ev = [0 for i in range(M)]
Ev[0] = ev1arrdeptime()
Ev[1] = ev1arrdeptime()
Date = '01.01.2014 '
Time = ':00'
Starttime =[0 for i in range(M)]
Departtime =[0 for i in range(M)]
for i in range(0,M):
    Starttime[i] = Date + Ev[i][1] +Time
    Departtime[i] = Date + Ev[i][2] +Time
    print('energy:',Ev[i][0])
    print('ev_a:',Starttime[i])
    print('ev_d:',Departtime[i])
    for item,val in thisdict.items():
        if item == Starttime[i]:
            ev_a[i] = val
        elif item == Departtime[i]:
            ev_d[i] = val
if ev_a[0] < ev_a[1]:        
    TS = [ev_a[0],ev_a[1]]  
    TD= [ev_d[0],ev_d[1]] 
    print('ev1 arrives before')
    print(TS)
    print(TD)
else:
    TS = [ev_a[1],ev_a[0]]   
    TD= [ev_d[1],ev_d[0]]
    print('ev2 arrives before')
    print(TS)
    print(TD)
    
    #TS = [ev1_a]
        #TD = [ev1_d]
    #M= i+1
    #parameterdy(M,96,Eint,Ereq,TS,TD)
    #M = 2
    

energy: 15.3
ev_a: 01.01.2014 13:15:00
ev_d: 01.01.2014 16:15:00
energy: 8.84
ev_a: 01.01.2014 16:45:00
ev_d: 01.01.2014 23:00:00
ev1 arrives before
[53, 67]
[65, 92]


In [42]:
#Test Dynamic Scheduling
#!/usr/bin/python
import csv
import numpy as np
from scipy import interpolate
from gurobipy import *
import random
import time
import sys
eff = 0.95
M=2
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []
thisdict = {
  '01.01.2014 00:00:00': 0,
  '01.01.2014 00:15:00': 1,
  '01.01.2014 00:30:00': 2
}
thisdict.update([ ('01.01.2014 00:45:00', 3) , ('01.01.2014 01:00:00', 4) , ('01.01.2014 01:15:00', 5) ,
                 ('01.01.2014 01:30:00', 6),('01.01.2014 01:45:00', 7),('01.01.2014 02:00:00', 8),
                 ('01.01.2014 02:15:00', 9),('01.01.2014 02:30:00', 10),('01.01.2014 02:45:00', 11),
                 ('01.01.2014 03:00:00', 12),('01.01.2014 03:15:00', 13),('01.01.2014 03:30:00', 14) , 
                 ('01.01.2014 03:45:00', 15) , ('01.01.2014 04:00:00', 16) , ('01.01.2014 04:15:00', 17),
                 ('01.01.2014 04:30:00', 18),('01.01.2014 04:45:00', 19),('01.01.2014 05:00:00', 20),
                 ('01.01.2014 05:15:00', 21),('01.01.2014 05:30:00', 22),('01.01.2014 05:45:00', 23),
                 ('01.01.2014 06:00:00', 24),('01.01.2014 06:15:00', 25) , ('01.01.2014 06:30:00', 26),
                 ('01.01.2014 06:45:00', 27),('01.01.2014 07:00:00', 28),('01.01.2014 07:15:00', 29),
                 ('01.01.2014 07:30:00', 30),('01.01.2014 07:45:00', 31),('01.01.2014 08:00:00', 32),
                 ('01.01.2014 08:15:00', 33),('01.01.2014 08:30:00', 34) , ('01.01.2014 08:45:00', 35) , 
                 ('01.01.2014 09:00:00', 36) , ('01.01.2014 09:15:00', 37),('01.01.2014 09:30:00', 38),
                 ('01.01.2014 09:45:00', 39),('01.01.2014 10:00:00', 40),('01.01.2014 10:15:00', 41),
                 ('01.01.2014 10:30:00', 42),('01.01.2014 10:45:00', 43),('01.01.2014 11:00:00', 44),
                 ('01.01.2014 11:15:00', 45) , ('01.01.2014 11:30:00', 46),('01.01.2014 11:45:00', 47),
                 ('01.01.2014 12:00:00', 48),('01.01.2014 12:15:00', 49),('01.01.2014 12:30:00', 50),
                 ('01.01.2014 12:45:00', 51),('01.01.2014 13:00:00', 52),('01.01.2014 13:15:00', 53),
                 ('01.01.2014 13:30:00', 54) , ('01.01.2014 13:45:00', 55) , ('01.01.2014 14:00:00', 56) , 
                 ('01.01.2014 14:15:00', 57),('01.01.2014 14:30:00', 58),('01.01.2014 14:45:00', 59),
                 ('01.01.2014 15:00:00', 60),('01.01.2014 15:15:00', 61),('01.01.2014 15:30:00', 62),
                 ('01.01.2014 15:45:00', 63),('01.01.2014 16:00:00', 64),('01.01.2014 16:15:00', 65) , 
                 ('01.01.2014 16:30:00', 66),('01.01.2014 16:45:00', 67),('01.01.2014 17:00:00', 68),
                 ('01.01.2014 17:15:00', 69),('01.01.2014 17:30:00', 70),('01.01.2014 17:45:00', 71),
                 ('01.01.2014 18:00:00', 72),('01.01.2014 18:15:00', 73),('01.01.2014 18:30:00', 74) , 
                 ('01.01.2014 18:45:00', 75) , ('01.01.2014 19:00:00', 76) , ('01.01.2014 19:15:00', 77),
                 ('01.01.2014 19:30:00', 78),('01.01.2014 19:45:00', 79),('01.01.2014 20:00:00', 80),
                 ('01.01.2014 20:15:00', 81),('01.01.2014 20:30:00', 82),('01.01.2014 20:45:00', 83),
                 ('01.01.2014 21:00:00',84),('01.01.2014 21:15:00', 85) , ('01.01.2014 21:30:00', 86),
                 ('01.01.2014 21:45:00', 87),('01.01.2014 22:00:00', 88),('01.01.2014 22:15:00', 89),
                 ('01.01.2014 22:30:00', 90),('01.01.2014 22:45:00', 91),('01.01.2014 23:00:00', 92),
                 ('01.01.2014 23:15:00', 93),('01.01.2014 23:30:00', 94) , ('01.01.2014 23:45:00', 95)])
def ev1arrdeptime():
    with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EV1sampledata.csv') as f:
        reader = csv.reader(f)
        header = next(reader)
        y = []
        for row in reader :
            ydist = float(row[0])
            yaDate = row[1]
            ydDate = row[2]
            y.append([ydist,yaDate,ydDate])
        ev1 =(random.choice(y))
    return(ev1);
def parameterdy( i, T, IE,RE,TS,TD):
    M= i
    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/household8_profile.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHP_a = float(row[0])
        HHP_b = float(row[1])
        HHP_c = float(row[2])
        HHQ_a = float(row[3])
        HHQ_b = float(row[4])
        HHQ_c = float(row[5])
        HH.append([HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PV_P = float(row[0])
        PV.append(PV_P)
    m = Model("dynamicscheduling")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    #Phase constraint
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
            
    #SOC user constraint
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            e[i][t] +=  p[i,t] * eff * 1/4  # Energy consumed by EV1   1/4 *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]
            te[i] += e[i][t]
            n = te[i]
        initial = IE[i] 
        required = RE 
        I_E = int(np.round(initial))
        R_E = int(np.round(required))
        m.addConstr(n >=0,name="SOC_minconstraint[%d]" %i)
        m.addConstr(n <=24,name="SOC_maxconstraint[%d]" %i)
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
    
    #power outside TS TD as 0
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))

    #Grid constraint
    for t in range(T):
        for i in range(M):
    #if i < M :
            pa[i][t] = a[i,t] * p[i,t]
            pb[i][t] = b[i,t] * p[i,t]
            pc[i][t] = c[i,t] * p[i,t]
            
            patot[t] += pa[i][t]
            pbtot[t] += pb[i][t]
            pctot[t] += pc[i][t]
        #print(pa[i][t])
        pahhpv[t] = patot[t] + HH[t][0] - PV[t]
        pbhhpv[t] = pbtot[t] + HH[t][1]
        pchhpv[t] = pctot[t] + HH[t][2]
   
        
        #pab[t] = patot[t] - pbtot[t]
        #pbc[t] = pbtot[t] - pctot[t]
        #pca[t] = pctot[t] - patot[t]
    
        pab[t] = pahhpv[t] - pbhhpv[t]
        pbc[t] = pbhhpv[t] - pchhpv[t]
        pca[t] = pchhpv[t] - pahhpv[t]
    
        m.addConstr(pab[t] <= 4.60, name="Power_A_B_maxconstraint")
        m.addConstr(pab[t] >= -4.60, name="Power_A_B_minconstraint")
        m.addConstr(pbc[t] <= 4.60, name="Power_B_C_maxconstraint")
        m.addConstr(pbc[t] >= -4.60, name="Power_B_C_minconstraint")
        m.addConstr(pca[t] <= 4.60, name="Power_C_A_maxconstraint")
        m.addConstr(pca[t] >= -4.60, name="Power_C_A_minconstraint")
    
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    Im_cost = C
    ST = STime[0]
    i = M - 1
    DT = DTime[i]
    for t in range(ST,DT):
        if pab[t] <= 2.3:
            Im_cost += 0.0
        elif pab[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pbc[t] <= 2.3:
            Im_cost += 0.0
        elif pbc[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pca[t] <= 2.3:
            Im_cost += 0.0
        elif pca[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
    #set objective:
    m.ModelSense = GRB.MINIMIZE
    Im_C = Im_cost
    #m.setObjective(C)
    m.setObjective(Im_C)

    m.write('testDynamicscheduling.lp')
    #m.write('testcost.lp')

    # Compute optimal solution
    m.optimize()
    m.write('testDynamicscheduling.sol')
    #print solution 

    for v in m.getVars():
        print(v.varName, v.x)
        
    print('Obj:', m.objVal)
    finalcost = m.objVal
         
       # elif i ==1:
       #     time.sleep(30)
       #     Ev2 = ev2adt() #ev2arrdeptime()
       #     print('ev2:',Ev2)
       #     print('ev2_a:',Ev2[1])
       #     print('ev2_d:',Ev2[2])
    return(finalcost);
M = 2
T = 96
#power=[0 for i in range(T)]
InitialEgy = [0 for i in range(M)] 
IEgy = [0 for i in range(M)] 
ev_a=[0 for i in range(M)]
ev_d=[0 for i in range(M)]
Ev = [0 for i in range(M)]
IE = [0 for i in range(M)]
RE = 17.0
Ev[0] = ev1arrdeptime()
Ev[1] = ev1arrdeptime()
Date = '01.01.2014 '
Time = ':00'
Starttime =[0 for i in range(M)]
Departtime =[0 for i in range(M)]
for i in range(0,M):
    Starttime[i] = Date + Ev[i][1] +Time
    Departtime[i] = Date + Ev[i][2] +Time
    print('energy:',Ev[i][0])
    IE[i] = RE - Ev[i][0]
    print('Initial energy:',IE[i])
    print('ev_a:',Starttime[i])
    print('ev_d:',Departtime[i])
    for item,val in thisdict.items():
        if item == Starttime[i]:
            ev_a[i] = val
        elif item == Departtime[i]:
            ev_d[i] = val
if ev_a[0] < ev_a[1]:        
    TS = [ev_a[0],ev_a[1]]  
    TD= [ev_d[0],ev_d[1]] 
    print('ev1 arrives before')
    print(TS)
    print(TD)
else:
    TS = [ev_a[1],ev_a[0]]   
    TD= [ev_d[1],ev_d[0]]
    print('ev2 arrives before')
    print(TS)
    print(TD)
if TS[0] < TS[1] and TS[1] < 96:
    if TS[1] < TD[0]:
        print('nothing')
        #print('inbetween')
        #Start = [TS[0]]
        #Depart = [TD[0]]
        #cost1=parameterdy(1,96,IE,RE,Start,Depart)

        #with open('testDynamicscheduling.sol', newline='') as f:
        #    reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        #    next(reader)            
        #    next(reader)
        #    lines = list(reader)

        #with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
        #    wr = csv.writer(myfile, delimiter=',')
        #    wr.writerows(lines)
            
        #with open('testDynamicscheduling1ev.csv') as csv_file:
        #    csv_reader = csv.reader(csv_file, delimiter=',')
        #    power =[]
        #    p = []
        #    for row in csv_reader:
        #        power_str = row[0]
        #        power_P = float(row[1])
        #        power.append([power_str,power_P])
        #    for i in range(len(power)):
        #        for j in range(TS[0],TS[1]):
        #            if power[i][0] == 'Power[%d,%d]' %(0,j):
        #                p.append(power[i][1])
        #    print(p)
        #    energy =[]
        #    pwr=[0 for i in range(T)]
        #    for i in range(len(p)):
        #        pwr[i] = p[i] *eff * 1/4
        #        energy.append(pwr[i])
        #        InitialEgy[0] += energy[i]
        #    print(energy)
        #    print(InitialEgy[0])
            
        #Start = [TS[1],TS[1]]
        #Depart = [TD[0],TD[1]]
        #IE[0] += InitialEgy[0]
        #cost2=parameterdy(2,96,IE,RE,Start,Depart)
        #with open('testDynamicscheduling.sol', newline='') as f:
        #    reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        #    next(reader)
        #    next(reader)
        #    lines = list(reader)

        #with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
        #    wr = csv.writer(myfile, delimiter=',')
        #    wr.writerows(lines)
    
        #f.close()
        #myfile.close()
        #finalcost = 2.43 + cost2
        #print('final optimal cost:',finalcost)
    elif TS[1] > TD[0]:
        if TD[0] < TS[0] and TD[1] > TS[1]:
            print('inbetween')
            Start = [TS[0]]
            Depart = [96]
            cost1=parameterdy(1,96,IE,RE,Start,Depart)
        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)            
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
            
        with open('testDynamicscheduling1ev.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            power =[]
            p = []
            for row in csv_reader:
                power_str = row[0]
                power_P = float(row[1])
                power.append([power_str,power_P])
            for i in range(len(power)):
                for j in range(TS[0],TS[1]):
                    if power[i][0] == 'Power[%d,%d]' %(0,j):
                        p.append(power[i][1])
            print(p)
            energy =[]
            pwr=[0 for i in range(T)]
            for i in range(len(p)):
                pwr[i] = p[i] *eff * 1/4
                energy.append(pwr[i])
                InitialEgy[0] += energy[i]
            print(energy)
            print(InitialEgy[0])
            
        Start = [TS[1],TS[1]]
        Depart = [96,TD[1]]
        IE[0] += InitialEgy[0]
        cost2=parameterdy(2,96,IE,RE,Start,Depart)
        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
            
        with open('testDynamicscheduling2evs.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            powr =[]
            po = []
            for row in csv_reader:
                powr_str = row[0]
                powr_P = float(row[1])
                powr.append([powr_str,powr_P])
            for i in range(len(powr)):
                for j in range(TS[1],TD[1]):
                    if powr[i][0] == 'Power[%d,%d]' %(0,j):
                        po.append(powr[i][1])
            print(po)
            egy =[]
            pwrs=[0 for i in range(T)]
            for i in range(len(po)):
                pwrs[i] = po[i] *eff * 1/4
                egy.append(pwrs[i])
                IEgy[0] += egy[i]
            print(egy)
            print(IEgy[0])
            
        Start = [0]
        Depart = [TD[0]]
        IE[0] += IEgy[0]    
        cost3=parameterdy(1,96,IE,RE,Start,Depart)
        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)            
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling1evfinal.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
        finalcost = cost1 + cost2 +cost3
        print('final optimal cost:',finalcost)
        f.close()
        myfile.close()
    else:
        exit

energy: 12.24
Initial energy: 4.76
ev_a: 01.01.2014 15:15:00
ev_d: 01.01.2014 08:45:00
energy: 9.35
Initial energy: 7.65
ev_a: 01.01.2014 17:15:00
ev_d: 01.01.2014 23:00:00
ev1 arrives before
[61, 69]
[35, 92]
inbetween
Optimize a model with 158 rows, 384 columns and 384 nonzeros
Model has 576 quadratic constraints
Variable types: 96 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [7e-02, 8e-02]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 1e+01]
  QRHS range       [3e-01, 9e+00]
Presolve removed 122 rows and 244 columns
Presolve time: 0.06s
Presolved: 876 rows, 560 columns, 2450 nonzeros
Variable types: 455 continuous, 105 integer (105 binary)

Root relaxation: objective 3.371108e+00, 335 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

Optimize a model with 336 rows, 768 columns and 768 nonzeros
Model has 576 quadratic constraints
Variable types: 192 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [7e-02, 8e-02]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 9e+00]
  QRHS range       [3e-01, 9e+00]
Presolve removed 284 rows and 568 columns
Presolve time: 0.07s
Presolved: 1114 rows, 800 columns, 3500 nonzeros
Variable types: 650 continuous, 150 integer (150 binary)

Root relaxation: objective 3.302049e+00, 453 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.30205    0   62          -    3.30205      -     -    0s
H    0     0                       3.5378851    3.30205  6.67%     -    0s
H    0     0                       3.4676858    3.30205  4.78%     -    0s
 

 308980 173365    3.44633   72   39    3.45154    3.41863  0.95%  18.0  245s
 319050 179654    3.45136   75   25    3.45154    3.41877  0.95%  18.1  250s
 326886 184729    3.43023   52   67    3.45154    3.41886  0.95%  18.1  255s
 336411 190844    3.42486   51   72    3.45154    3.41897  0.94%  18.1  260s
 343958 195580    3.44232   65   47    3.45154    3.41905  0.94%  18.1  265s
 352467 201216    3.44033   68   52    3.45154    3.41913  0.94%  18.1  270s
 360921 206629    3.43146   53   73    3.45154    3.41923  0.94%  18.1  275s
 370737 212857     cutoff   65         3.45154    3.41933  0.93%  18.1  280s
 378276 217720    3.42290   49   74    3.45154    3.41938  0.93%  18.1  285s
 388208 223950    3.44268   65   47    3.45154    3.41948  0.93%  18.1  290s
 395977 229007     cutoff   78         3.45154    3.41957  0.93%  18.1  295s
 406178 235355    3.44336   67   45    3.45154    3.41967  0.92%  18.1  300s
 414098 240243    3.44453   64   52    3.45154    3.41974  0.92%  18.1  305s

 1198665 701538    3.44898   67   52    3.45154    3.42390  0.80%  18.5  781s
 1206393 705837    3.44938   72   38    3.45154    3.42392  0.80%  18.5  785s
 1215877 711055    3.45008   61   55    3.45154    3.42396  0.80%  18.6  791s
 1224001 715499    3.44374   66   45    3.45154    3.42398  0.80%  18.6  795s
 1231962 719897    3.44857   72   40    3.45154    3.42401  0.80%  18.6  800s
 1241890 725300    3.45105   64   45    3.45154    3.42403  0.80%  18.6  805s
 1249901 729650    3.43782   62   60    3.45154    3.42406  0.80%  18.6  810s
 1257675 733927    3.45082   66   47    3.45154    3.42408  0.80%  18.6  815s
 1265366 738154    3.42958   57   66    3.45154    3.42411  0.79%  18.6  820s
 1275664 743788    3.44854   70   39    3.45154    3.42414  0.79%  18.6  825s
 1283060 747806    3.44455   59   53    3.45154    3.42416  0.79%  18.6  830s
 1293366 753564    3.43463   53   73    3.45154    3.42419  0.79%  18.6  836s
 1301334 758052    3.45028   66   47    3.45154    3.42421  0.79

 2060401 1171496     cutoff   68         3.45154    3.42579  0.75%  19.0 1305s
 2070289 1176787    3.44040   61   53    3.45154    3.42580  0.75%  19.0 1311s
 2072367 1177890     cutoff   60         3.45154    3.42581  0.75%  19.0 1315s
 2078217 1181079     cutoff   68         3.45154    3.42582  0.75%  19.0 1320s
 2086346 1185584    3.45021   75   37    3.45154    3.42583  0.74%  19.0 1326s
 2092454 1188827    3.43976   60   58    3.45154    3.42583  0.74%  19.0 1330s
 2100320 1193108    3.44839   67   45    3.45154    3.42585  0.74%  19.0 1336s
 2106249 1196284    3.43350   56   65    3.45154    3.42586  0.74%  19.0 1340s
 2114240 1200647    3.45037   74   28    3.45154    3.42587  0.74%  19.0 1346s
 2119879 1203604    3.45116   74   32    3.45154    3.42588  0.74%  19.0 1350s
 2121866 1204682    3.43884   58   60    3.45154    3.42588  0.74%  19.0 1355s
 2127882 1207997    3.43286   57   68    3.45154    3.42589  0.74%  19.0 1361s
 2133885 1211314    3.43597   51   73    3.45154    

...

 3629275 2023810    3.43461   52   71    3.45154    3.42734  0.70%  19.4 2875s
 3632994 2025797    3.44609   65   46    3.45154    3.42734  0.70%  19.4 2880s
 3637185 2028017    3.42917   56   65    3.45154    3.42735  0.70%  19.4 2885s
 3641289 2030197    3.43989   55   70    3.45154    3.42735  0.70%  19.4 2890s
 3647260 2033415    3.44995   72   40    3.45154    3.42735  0.70%  19.4 2897s
 3650763 2035257    3.44667   67   46    3.45154    3.42735  0.70%  19.4 2902s
 3654903 2037596    3.44808   67   46    3.45154    3.42736  0.70%  19.4 2907s
 3658935 2039713    3.44363   57   62    3.45154    3.42736  0.70%  19.4 2912s
 3663012 2042013    3.44912   78   33    3.45154    3.42736  0.70%  19.4 2917s
 3667117 2044173    3.43702   58   62    3.45154    3.42736  0.70%  19.4 2921s
 3671107 2046279     cutoff   69         3.45154    3.42737  0.70%  19.4 2926s
 3674882 2048224    3.44090   60   56    3.45154    3.42737  0.70%  19.4 2931s
 3678761 2050293     cutoff   72         3.45154    

 4023655 2234774    3.44663   68   38    3.45154    3.42758  0.69%  19.5 3400s
 4027938 2236956     cutoff   72         3.45154    3.42758  0.69%  19.5 3405s
 4032090 2239176    3.43579   58   65    3.45154    3.42758  0.69%  19.5 3411s
 4036104 2241382    3.45065   75   31    3.45154    3.42759  0.69%  19.5 3416s
 4040042 2243334    3.44195   58   55    3.45154    3.42759  0.69%  19.5 3421s
 4044066 2245467    3.45119   76   33    3.45154    3.42759  0.69%  19.5 3427s
 4047911 2247464    3.45024   68   41    3.45154    3.42759  0.69%  19.5 3432s
 4049830 2248484    3.44174   61   54    3.45154    3.42759  0.69%  19.5 3435s
 4053941 2250719    3.44265   57   61    3.45154    3.42760  0.69%  19.5 3440s
 4057990 2252901    3.42900   56   70    3.45154    3.42760  0.69%  19.5 3445s
 4062137 2254999    3.44712   66   41    3.45154    3.42760  0.69%  19.5 3451s
 4066010 2257160    3.45119   73   39    3.45154    3.42760  0.69%  19.5 3456s
 4069828 2259223    3.44491   60   52    3.45154    

 4391535 2429716    3.44950   70   38    3.45154    3.42779  0.69%  19.5 3925s
 4395476 2431860    3.44778   62   46    3.45154    3.42779  0.69%  19.5 3931s
 4399726 2434110    3.42927   57   65    3.45154    3.42780  0.69%  19.5 3936s
 4403310 2436022    3.44401   68   47    3.45154    3.42780  0.69%  19.5 3942s
 4405513 2437179    3.43262   53   67    3.45154    3.42780  0.69%  19.5 3945s
 4409501 2439334    3.42941   55   62    3.45154    3.42780  0.69%  19.5 3951s
 4413555 2441473    3.43515   53   66    3.45154    3.42780  0.69%  19.5 3957s
 4415425 2442524     cutoff   70         3.45154    3.42780  0.69%  19.6 3960s
 4419380 2444545    3.43287   53   67    3.45154    3.42781  0.69%  19.6 3965s
 4423254 2446605    3.43262   59   61    3.45154    3.42781  0.69%  19.6 3971s
 4427239 2448679     cutoff   70         3.45154    3.42781  0.69%  19.6 3977s
 4429272 2449756    3.44980   68   45    3.45154    3.42781  0.69%  19.6 3980s
 4432922 2451655    3.43789   55   65    3.45154    

 4718984 2600337    3.43520   56   63    3.45154    3.42797  0.68%  19.6 4450s
 4723064 2602495    3.44858   72   40    3.45154    3.42797  0.68%  19.6 4456s
 4725017 2603534    3.44568   70   48    3.45154    3.42797  0.68%  19.6 4460s
 4729026 2605677     cutoff   71         3.45154    3.42797  0.68%  19.6 4466s
 4733002 2607685    3.44370   64   48    3.45154    3.42798  0.68%  19.6 4473s
 4734925 2608691    3.44130   60   69    3.45154    3.42798  0.68%  19.6 4476s
 4738972 2610877     cutoff   77         3.45154    3.42798  0.68%  19.6 4483s
 4740977 2611930    3.44642   62   50    3.45154    3.42798  0.68%  19.6 4486s
 4744857 2613917    3.43179   56   69    3.45154    3.42798  0.68%  19.6 4492s
 4746734 2614845    3.44380   59   55    3.45154    3.42798  0.68%  19.6 4496s
 4750685 2617065     cutoff   78         3.45154    3.42799  0.68%  19.6 4502s
 4752710 2618076    3.43070   56   69    3.45154    3.42799  0.68%  19.6 4505s
 4756688 2620118    3.44842   67   47    3.45154    

GurobiError: Out of memory

In [43]:
#Test dayahead scheduling
#!/usr/bin/python
import csv
import numpy as np
from datetime import time,datetime,date
from scipy import interpolate
from gurobipy import *

eff = 0.95
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []
thisdict = {
  '01.01.2014 00:00:00': 0,
  '01.01.2014 00:15:00': 1,
  '01.01.2014 00:30:00': 2
}
thisdict.update([ ('01.01.2014 00:45:00', 3) , ('01.01.2014 01:00:00', 4) , ('01.01.2014 01:15:00', 5) ,
                 ('01.01.2014 01:30:00', 6),('01.01.2014 01:45:00', 7),('01.01.2014 02:00:00', 8),
                 ('01.01.2014 02:15:00', 9),('01.01.2014 02:30:00', 10),('01.01.2014 02:45:00', 11),
                 ('01.01.2014 03:00:00', 12),('01.01.2014 03:15:00', 13),('01.01.2014 03:30:00', 14) , 
                 ('01.01.2014 03:45:00', 15) , ('01.01.2014 04:00:00', 16) , ('01.01.2014 04:15:00', 17),
                 ('01.01.2014 04:30:00', 18),('01.01.2014 04:45:00', 19),('01.01.2014 05:00:00', 20),
                 ('01.01.2014 05:15:00', 21),('01.01.2014 05:30:00', 22),('01.01.2014 05:45:00', 23),
                 ('01.01.2014 06:00:00', 24),('01.01.2014 06:15:00', 25) , ('01.01.2014 06:30:00', 26),
                 ('01.01.2014 06:45:00', 27),('01.01.2014 07:00:00', 28),('01.01.2014 07:15:00', 29),
                 ('01.01.2014 07:30:00', 30),('01.01.2014 07:45:00', 31),('01.01.2014 08:00:00', 32),
                 ('01.01.2014 08:15:00', 33),('01.01.2014 08:30:00', 34) , ('01.01.2014 08:45:00', 35) , 
                 ('01.01.2014 09:00:00', 36) , ('01.01.2014 09:15:00', 37),('01.01.2014 09:30:00', 38),
                 ('01.01.2014 09:45:00', 39),('01.01.2014 10:00:00', 40),('01.01.2014 10:15:00', 41),
                 ('01.01.2014 10:30:00', 42),('01.01.2014 10:45:00', 43),('01.01.2014 11:00:00', 44),
                 ('01.01.2014 11:15:00', 45) , ('01.01.2014 11:30:00', 46),('01.01.2014 11:45:00', 47),
                 ('01.01.2014 12:00:00', 48),('01.01.2014 12:15:00', 49),('01.01.2014 12:30:00', 50),
                 ('01.01.2014 12:45:00', 51),('01.01.2014 13:00:00', 52),('01.01.2014 13:15:00', 53),
                 ('01.01.2014 13:30:00', 54) , ('01.01.2014 13:45:00', 55) , ('01.01.2014 14:00:00', 56) , 
                 ('01.01.2014 14:15:00', 57),('01.01.2014 14:30:00', 58),('01.01.2014 14:45:00', 59),
                 ('01.01.2014 15:00:00', 60),('01.01.2014 15:15:00', 61),('01.01.2014 15:30:00', 62),
                 ('01.01.2014 15:45:00', 63),('01.01.2014 16:00:00', 64),('01.01.2014 16:15:00', 65) , 
                 ('01.01.2014 16:30:00', 66),('01.01.2014 16:45:00', 67),('01.01.2014 17:00:00', 68),
                 ('01.01.2014 17:15:00', 69),('01.01.2014 17:30:00', 70),('01.01.2014 17:45:00', 71),
                 ('01.01.2014 18:00:00', 72),('01.01.2014 18:15:00', 73),('01.01.2014 18:30:00', 74) , 
                 ('01.01.2014 18:45:00', 75) , ('01.01.2014 19:00:00', 76) , ('01.01.2014 19:15:00', 77),
                 ('01.01.2014 19:30:00', 78),('01.01.2014 19:45:00', 79),('01.01.2014 20:00:00', 80),
                 ('01.01.2014 20:15:00', 81),('01.01.2014 20:30:00', 82),('01.01.2014 20:45:00', 83),
                 ('01.01.2014 21:00:00',84),('01.01.2014 21:15:00', 85) , ('01.01.2014 21:30:00', 86),
                 ('01.01.2014 21:45:00', 87),('01.01.2014 22:00:00', 88),('01.01.2014 22:15:00', 89),
                 ('01.01.2014 22:30:00', 90),('01.01.2014 22:45:00', 91),('01.01.2014 23:00:00', 92),
                 ('01.01.2014 23:15:00', 93),('01.01.2014 23:30:00', 94) , ('01.01.2014 23:45:00', 95)])
def parameterda( M, T, Eint,Ereq,TS,TD):

    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/HH_profile_Scheduling.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHDate = row[0]
        HHP_a = float(row[1])
        HHP_b = float(row[2])
        HHP_c = float(row[3])
        HHQ_a = float(row[4])
        HHQ_b = float(row[5])
        HHQ_c = float(row[6])
        HH.append([HHDate,HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile_Scheduling.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PVDate = row[0]
        PV_P = float(row[1])
        PV.append([PVDate,PV_P])
    m = Model("dayahead")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    #Phase constraint
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
            
    #SOC user constraint
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            e[i][t] +=  p[i,t] * eff * 1/4  # Energy consumed by EV1   1/4 *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]
            te[i] += e[i][t]
            n = te[i]
        initial = IE[i] 
        required = RE 
        I_E = int(np.round(initial))
        R_E = int(np.round(required))
        m.addConstr(n >=0,name="SOC_minconstraint[%d]" %i)
        m.addConstr(n <=24,name="SOC_maxconstraint[%d]" %i)
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
    
    #power outside TS TD as 0
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))

    #Grid constraint
    for t in range(T):
        #if HH[i][0] == atime:
        for i in range(M):
    #if i < M :
            pa[i][t] = a[i,t] * p[i,t]
            pb[i][t] = b[i,t] * p[i,t]
            pc[i][t] = c[i,t] * p[i,t]
            
            patot[t] += pa[i][t]
            pbtot[t] += pb[i][t]
            pctot[t] += pc[i][t]
        #print(pa[i][t])
        pahhpv[t] = patot[t] + HH[t][1] - PV[t][1]
        pbhhpv[t] = pbtot[t] + HH[t][2]
        pchhpv[t] = pctot[t] + HH[t][3]
   
    
        pab[t] = pahhpv[t] - pbhhpv[t]
        pbc[t] = pbhhpv[t] - pchhpv[t]
        pca[t] = pchhpv[t] - pahhpv[t]
    
        m.addConstr(pab[t] <= 4.60, name="Power_A_B_maxconstraint")
        m.addConstr(pab[t] >= -4.60, name="Power_A_B_minconstraint")
        m.addConstr(pbc[t] <= 4.60, name="Power_B_C_maxconstraint")
        m.addConstr(pbc[t] >= -4.60, name="Power_B_C_minconstraint")
        m.addConstr(pca[t] <= 4.60, name="Power_C_A_maxconstraint")
        m.addConstr(pca[t] >= -4.60, name="Power_C_A_minconstraint")
    
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    Im_cost = C
    ST = STime[0]
    i = M - 1
    DT = DTime[i]
    for t in range(ST,DT):
        if pab[t] <= 2.3:
            Im_cost += 0.0
        elif pab[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pbc[t] <= 2.3:
            Im_cost += 0.0
        elif pbc[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pca[t] <= 2.3:
            Im_cost += 0.0
        elif pca[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
    
    #set objective:
    m.ModelSense = GRB.MINIMIZE
    Im_C = Im_cost
    #m.setObjective(C)
    m.setObjective(Im_C)

    m.write('test1dayahead.lp')
    #m.write('testcost.lp')

    # Compute optimal solution
    m.optimize()
    m.write('test1dayahead.sol')
    #print solution 

    for v in m.getVars():
        print(v.varName, v.x)
    
    print('Obj:', m.objVal)
    
    with open('test1dayahead.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('test1dayahead.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
# do IIS
#    print('The model is infeasible; computing IIS')
#    m.computeIIS()
#    if m.IISMinimal:
#      print('IIS is minimal\n')
#    else:
#      print('IIS is not minimal\n')
#    print('\nThe following constraint(s) cannot be satisfied:')
#    for c in m.getConstrs():
#        if c.IISConstr:
#             print('%s' % c.constrName)
    return;
M=2
#Eint = [30,20];
#Ereq = [80,80];
atime =[0 for t in range(M)]
dtime =[0 for t in range(M)]
IE = [0 for i in range(M)]
EV1_a = datetime(2014, 1, 1, 13, 30)
atime[0] = EV1_a.strftime("%d.%m.%Y %H:%M:%S")
EV2_a = datetime(2014, 1, 1, 14, 45)
atime[1] = EV2_a.strftime("%d.%m.%Y %H:%M:%S")
EV1_d = datetime(2014, 1, 1, 17, 15)
dtime[0] = EV1_d.strftime("%d.%m.%Y %H:%M:%S")
EV2_d = datetime(2014, 1, 1, 23, 00)
dtime[1] = EV2_d.strftime("%d.%m.%Y %H:%M:%S")
for i in range(0,M):
    for item,val in thisdict.items():
        if item == atime[i]:
            ev_a[i] = val
        elif item == dtime[i]:
            ev_d[i] = val
TS = [ev_a[0],ev_a[1]]
TD = [ev_d[0],ev_d[1]]
RE = 17.0
IE =[7.48,3.74]
parameterda(2,96,IE,RE,TS,TD)

Optimize a model with 342 rows, 768 columns and 864 nonzeros
Model has 576 quadratic constraints
Variable types: 192 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [7e-02, 8e-02]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [3e-01, 9e+00]
Presolve removed 292 rows and 576 columns
Presolve time: 0.11s
Presolved: 1142 rows, 768 columns, 3360 nonzeros
Variable types: 624 continuous, 144 integer (144 binary)

Root relaxation: objective 6.759456e+00, 481 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.75946    0   87          -    6.75946      -     -    0s
H    0     0                       6.9842849    6.75946  3.22%     -    0s
H    0     0                       6.9288925    6.75946  2.45%     -    0s
H

 514557 141088     cutoff   39         6.85878    6.85315  0.08%  16.9  360s
 518447 141878    6.85660   41   11    6.85878    6.85316  0.08%  16.9  365s
 526174 143462     cutoff   44         6.85878    6.85317  0.08%  16.9  370s
 533960 145147    6.85389   32   12    6.85878    6.85320  0.08%  16.9  375s
 542018 146909    6.85854   44    7    6.85878    6.85320  0.08%  16.9  380s
 550097 148478    6.85624   41    8    6.85878    6.85322  0.08%  16.9  386s
 557961 150062    6.85470   33   24    6.85878    6.85324  0.08%  16.9  390s
 565517 151529     cutoff   43         6.85878    6.85326  0.08%  16.9  396s
 571124 152570    6.85539   40   12    6.85878    6.85328  0.08%  16.8  400s
 577181 153891    6.85788   37    9    6.85878    6.85328  0.08%  16.8  405s
 582904 155148     cutoff   41         6.85878    6.85329  0.08%  16.8  410s
 590695 156778     cutoff   44         6.85878    6.85330  0.08%  16.8  415s
 598287 158372    6.85399   37   12    6.85878    6.85332  0.08%  16.8  421s

 1434657 282787    6.85439   30   12    6.85878    6.85436  0.06%  16.3  890s
 1446423 283627    6.85659   40    7    6.85878    6.85437  0.06%  16.3  895s
 1457764 284617    6.85556   39   10    6.85878    6.85438  0.06%  16.3  900s
 1469588 285654    6.85784   39   10    6.85878    6.85440  0.06%  16.3  905s
 1481248 286572    6.85443   33   12    6.85878    6.85441  0.06%  16.3  910s
 1487994 287206    6.85830   42   12    6.85878    6.85442  0.06%  16.3  915s
 1500136 288148     cutoff   43         6.85878    6.85442  0.06%  16.3  920s
 1509775 289150    6.85717   39   10    6.85878    6.85443  0.06%  16.3  925s
 1521494 290283    6.85534   33   11    6.85878    6.85445  0.06%  16.3  930s
 1533161 291170     cutoff   40         6.85878    6.85445  0.06%  16.3  935s
 1540908 291880     cutoff   44         6.85878    6.85446  0.06%  16.3  940s
 1550504 292887    6.85691   42    9    6.85878    6.85447  0.06%  16.3  945s
 1558230 293711     cutoff   45         6.85878    6.85447  0.06

 2709105 316447    6.85612   38   16    6.85878    6.85551  0.05%  16.6 1420s
 2720301 315833    6.85858   41   12    6.85878    6.85552  0.05%  16.6 1425s
 2731051 315117    6.85553   31   20    6.85878    6.85553  0.05%  16.6 1430s
 2742875 314356    6.85634   41   12    6.85878    6.85554  0.05%  16.6 1435s
 2754485 313922    6.85770   35   12    6.85878    6.85555  0.05%  16.6 1441s
 2760732 313264    6.85717   33   17    6.85878    6.85556  0.05%  16.6 1445s
 2770795 312608     cutoff   39         6.85878    6.85556  0.05%  16.6 1450s
 2782273 311811 infeasible   41         6.85878    6.85558  0.05%  16.6 1455s
 2793826 311027    6.85801   43    9    6.85878    6.85559  0.05%  16.6 1460s
 2807170 310102     cutoff   38         6.85878    6.85561  0.05%  16.6 1465s
 2820299 309129    6.85813   40    9    6.85878    6.85561  0.05%  16.6 1470s
 2831935 308158     cutoff   42         6.85878    6.85563  0.05%  16.6 1475s
 2845044 307101     cutoff   39         6.85878    6.85564  0.05

 3864073 149336    6.85829   35   13    6.85878    6.85708  0.02%  16.9 1950s
 3873322 146457    6.85808   34   10    6.85878    6.85710  0.02%  16.9 1955s
 3884464 143284    6.85713   34   13    6.85878    6.85713  0.02%  16.9 1960s
 3895425 140148    6.85719   31   20    6.85878    6.85716  0.02%  16.9 1965s
 3906450 137010    6.85724   34   16    6.85878    6.85719  0.02%  16.9 1970s
 3917113 133826    6.85722   38   10    6.85878    6.85722  0.02%  17.0 1975s
 3928636 130297    6.85821   38   12    6.85878    6.85725  0.02%  17.0 1980s
 3940048 126754    6.85745   37   14    6.85878    6.85729  0.02%  17.0 1985s
 3950985 123202    6.85847   39   11    6.85878    6.85732  0.02%  17.0 1990s
 3960902 119956    6.85735   38   12    6.85878    6.85735  0.02%  17.0 1995s
 3971647 116377    6.85751   34   12    6.85878    6.85739  0.02%  17.0 2000s
 3979574 113798    6.85768   37   14    6.85878    6.85742  0.02%  17.0 2005s
 3990090 110214    6.85744   40    8    6.85878    6.85744  0.02

Power[1,39] 0.0
Power[1,40] 0.0
Power[1,41] 0.0
Power[1,42] 0.0
Power[1,43] 0.0
Power[1,44] 0.0
Power[1,45] 0.0
Power[1,46] 0.0
Power[1,47] 0.0
Power[1,48] 0.0
Power[1,49] 0.0
Power[1,50] 0.0
Power[1,51] 0.0
Power[1,52] 0.0
Power[1,53] 0.0
Power[1,54] 0.0
Power[1,55] 0.0
Power[1,56] 0.0
Power[1,57] 0.0
Power[1,58] 0.0
Power[1,59] 5.2179998614241745
Power[1,60] 4.69099977070625
Power[1,61] 6.007999766832442
Power[1,62] 4.865999770303069
Power[1,63] 4.684999770778584
Power[1,64] 7.1999995659458795
Power[1,65] 7.199999563947567
Power[1,66] 4.7999997037388775
Power[1,67] 7.199999565353241
Power[1,68] 7.199999570976973
Power[1,69] 4.7059992687016114
Power[1,70] 4.683999211591119
Power[1,71] 4.624999263380141
Power[1,72] -4.574676634487893
Power[1,73] -4.2876281980321345
Power[1,74] -4.244450320112492
Power[1,75] -4.279391898693802
Power[1,76] -4.652999216753489
Power[1,77] -4.785999175323374
Power[1,78] -4.9799989283224715
Power[1,79] -4.897999199814054
Power[1,80] -4.975999835301089
Power[

In [9]:
#Sample set
import random
import csv
dicti = {
  '00:00': 0,
  '00:15': 1,
  '00:30': 2
}
dicti.update([ ('00:45', 3) , ('01:00', 4) , ('01:15', 5) ,
                 ('01:30', 6),('01:45', 7),('02:00', 8),
                 ('02:15', 9),('02:30', 10),('02:45', 11),
                 ('03:00', 12),('03:15', 13),('03:30', 14) , 
                 ('03:45', 15) , ('04:00', 16) , ('04:15', 17),
                 ('04:30', 18),('04:45', 19),('05:00', 20),
                 ('05:15', 21),('05:30', 22),('05:45', 23),
                 ('06:00', 24),('06:15', 25) , ('06:30', 26),
                 ('06:45', 27),('07:00', 28),('07:15', 29),
                 ('07:30', 30),('07:45', 31),('08:00', 32),
                 ('08:15', 33),('08:30', 34) , ('08:45', 35) , 
                 ('09:00', 36) , ('09:15', 37),('09:30', 38),
                 ('09:45', 39),('10:00', 40),('10:15', 41),
                 ('10:30', 42),('10:45', 43),('11:00', 44),
                 ('11:15', 45) , ('11:30', 46),('11:45', 47),
                 ('12:00', 48),('12:15', 49),('12:30', 50),
                 ('12:45', 51),('13:00', 52),('13:15', 53),
                 ('13:30', 54) , ('13:45', 55) , ('14:00', 56) , 
                 ('14:15', 57),('14:30', 58),('14:45', 59),
                 ('15:00', 60),('15:15', 61),('15:30', 62),
                 ('15:45', 63),('16:00', 64),('16:15', 65) , 
                 ('16:30', 66),('16:45', 67),('17:00', 68),
                 ('17:15', 69),('17:30', 70),('17:45', 71),
                 ('18:00', 72),('18:15', 73),('18:30', 74) , 
                 ('18:45', 75) , ('19:00', 76) , ('19:15', 77),
                 ('19:30', 78),('19:45', 79),('20:00', 80),
                 ('20:15', 81),('20:30', 82),('20:45', 83),
                 ('21:00',84),('21:15', 85) , ('21:30', 86),
                 ('21:45', 87),('22:00', 88),('22:15', 89),
                 ('22:30', 90),('22:45', 91),('23:00', 92),
                 ('23:15', 93),('23:30', 94) , ('23:45', 95)])
with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/Arr_Dep_EV.csv', newline='') as f:
    reader = csv.reader(f)
    #reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
    header = next(reader)
    y = []
    for row in reader :
            ydist = row[0]
            yaDate = row[1]
            ydDate = row[2]
            y.append([ydist,yaDate,ydDate])

    ev1 =(random.choice(y))
    ev2 =(random.choice(y))
    print(ev1)
    print(ev2)
    for item,val in dicti.items():
        if item == ev1[1]:
            ev1_a = val
        elif item == ev1[2]:
            ev1_d = val
        elif item == ev2[1]:
            ev2_a = val
        elif item == ev2[2]:
            ev2_d = val
    if ev1_a < ev2_a and ev2_a < ev1_d and ev1_d < ev2_d:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
        #print('ev2 between ev1a and ev1d')
    elif ev2_a < ev1_a and ev1_a < ev2_d and ev2_d < ev1_d:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
        #print('ev1 between ev2a and ev2d')
    elif ev1_a < ev2_a and ev2_a < ev2_d and ev1_d < ev1_a:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
        #print('ev2 between ev1a and ev1d')
    elif ev2_a < ev1_a and ev1_a < ev1_d and ev2_d < ev2_a:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
    elif ev1_d < ev2_d and ev2_d < ev1_a and ev1_a < ev2_a:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
        #print('ev2 between ev1a and ev1d')
    elif ev2_d < ev1_d and ev1_d < ev2_a and ev2_a < ev1_a:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
        #print('ev1 between ev2a and ev2d')
#with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'w', newline='') as myfile:
#    wr = csv.writer(myfile, delimiter=',')
#    wr.writerow(ev1)
    


['18', '14:15', '17:00']
['90', '11:45', '12:00']


In [11]:
import csv
from scipy import interpolate
distance =[]
with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv') as file:
    reader = csv.reader(file)
    #header = next(reader)
    x = []
    for row in reader :
        xdist = float(row[0])
        xaDate = row[1]
        xdDate = row[2]
        x.append([xdist,xaDate,xdDate])
    #ev2 =(random.choice(x)) 
    for i in range(len(x)):
        #print(x[i][0])
        distance.append(x[i][0])
print(distance)
Distance = [0,100]    #Distance given /100 km
Energy = [0,17]  #Energy in kwh
f = interpolate.interp1d(Distance,Energy)
for i in range(len(distance)):
    evdistance = distance[i] 
    Energyeq = f(evdistance)
    print('Energy equivalent:',Energyeq)

[15.64, 8.755, 12.24, 6.494, 7.14, 3.91, 7.99, 14.28, 7.31, 11.56, 12.648, 9.52, 5.44, 6.528, 5.95, 10.71, 84.0, 11.05, 13.05, 7.14]
Energy equivalent: 2.6588000000000003
Energy equivalent: 1.4883500000000003
Energy equivalent: 2.0808
Energy equivalent: 1.10398
Energy equivalent: 1.2138
Energy equivalent: 0.6647000000000001
Energy equivalent: 1.3583
Energy equivalent: 2.4276
Energy equivalent: 1.2427
Energy equivalent: 1.9652000000000003
Energy equivalent: 2.15016
Energy equivalent: 1.6184
Energy equivalent: 0.9248000000000002
Energy equivalent: 1.10976
Energy equivalent: 1.0115
Energy equivalent: 1.8207000000000002
Energy equivalent: 14.280000000000001
Energy equivalent: 1.8785000000000003
Energy equivalent: 2.2185
Energy equivalent: 1.2138


In [6]:
import csv
M = 2
energycost1 = [0 for i in range(M)]
energy = [1.112212500000004, 1.0927375000000057, 1.1058000000000037, 1.1012875000000057, 1.1029500000000048, 1.0989125000000053, 1.117675000000006, 1.112450000000005]
path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
file2 = open(path2, newline='')
reader2 = csv.reader(file2)
header2 = next(reader2)
marketcost = []
for row in reader2 :
    mcost = float(row[0])
    marketcost.append(mcost)
cost = [j/100 for j in marketcost]
i = 0
for t in range(63,71):
    print('market cost',cost[t])
    energycost1[0] +=  energy[i] * cost[t] #energycost1[0] +
    print('energy',energy[i])
    print('energy cost',energycost1[0])
    i = i+1
print('total cost',energycost1[0])

market cost 0.3044496
energy 1.112212500000004
energy cost 0.33861265074000124
market cost 0.3063536
energy 1.0927375000000057
energy cost 0.673376717720003
market cost 0.3063536
energy 1.1058000000000037
energy cost 1.012142528600004
market cost 0.3063536
energy 1.1012875000000057
energy cost 1.3495259188600057
market cost 0.3063536
energy 1.1029500000000048
energy cost 1.6874186219800071
market cost 0.3101735
energy 1.0989125000000053
energy cost 2.028272158298759
market cost 0.3101735
energy 1.117675000000006
energy cost 2.374945324911261
market cost 0.3101735
energy 1.112450000000005
energy cost 2.7199978349862626
total cost 2.7199978349862626


In [ ]:
#Test Dynamic Scheduling
#!/usr/bin/python
import csv
import numpy as np
from scipy import interpolate
from gurobipy import *
import random
import time
import sys
eff = 0.95
M=2
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []
thisdict = {
  '01.01.2014 00:00:00': 0,
  '01.01.2014 00:15:00': 1,
  '01.01.2014 00:30:00': 2
}
thisdict.update([ ('01.01.2014 00:45:00', 3) , ('01.01.2014 01:00:00', 4) , ('01.01.2014 01:15:00', 5) ,
                 ('01.01.2014 01:30:00', 6),('01.01.2014 01:45:00', 7),('01.01.2014 02:00:00', 8),
                 ('01.01.2014 02:15:00', 9),('01.01.2014 02:30:00', 10),('01.01.2014 02:45:00', 11),
                 ('01.01.2014 03:00:00', 12),('01.01.2014 03:15:00', 13),('01.01.2014 03:30:00', 14) , 
                 ('01.01.2014 03:45:00', 15) , ('01.01.2014 04:00:00', 16) , ('01.01.2014 04:15:00', 17),
                 ('01.01.2014 04:30:00', 18),('01.01.2014 04:45:00', 19),('01.01.2014 05:00:00', 20),
                 ('01.01.2014 05:15:00', 21),('01.01.2014 05:30:00', 22),('01.01.2014 05:45:00', 23),
                 ('01.01.2014 06:00:00', 24),('01.01.2014 06:15:00', 25) , ('01.01.2014 06:30:00', 26),
                 ('01.01.2014 06:45:00', 27),('01.01.2014 07:00:00', 28),('01.01.2014 07:15:00', 29),
                 ('01.01.2014 07:30:00', 30),('01.01.2014 07:45:00', 31),('01.01.2014 08:00:00', 32),
                 ('01.01.2014 08:15:00', 33),('01.01.2014 08:30:00', 34) , ('01.01.2014 08:45:00', 35) , 
                 ('01.01.2014 09:00:00', 36) , ('01.01.2014 09:15:00', 37),('01.01.2014 09:30:00', 38),
                 ('01.01.2014 09:45:00', 39),('01.01.2014 10:00:00', 40),('01.01.2014 10:15:00', 41),
                 ('01.01.2014 10:30:00', 42),('01.01.2014 10:45:00', 43),('01.01.2014 11:00:00', 44),
                 ('01.01.2014 11:15:00', 45) , ('01.01.2014 11:30:00', 46),('01.01.2014 11:45:00', 47),
                 ('01.01.2014 12:00:00', 48),('01.01.2014 12:15:00', 49),('01.01.2014 12:30:00', 50),
                 ('01.01.2014 12:45:00', 51),('01.01.2014 13:00:00', 52),('01.01.2014 13:15:00', 53),
                 ('01.01.2014 13:30:00', 54) , ('01.01.2014 13:45:00', 55) , ('01.01.2014 14:00:00', 56) , 
                 ('01.01.2014 14:15:00', 57),('01.01.2014 14:30:00', 58),('01.01.2014 14:45:00', 59),
                 ('01.01.2014 15:00:00', 60),('01.01.2014 15:15:00', 61),('01.01.2014 15:30:00', 62),
                 ('01.01.2014 15:45:00', 63),('01.01.2014 16:00:00', 64),('01.01.2014 16:15:00', 65) , 
                 ('01.01.2014 16:30:00', 66),('01.01.2014 16:45:00', 67),('01.01.2014 17:00:00', 68),
                 ('01.01.2014 17:15:00', 69),('01.01.2014 17:30:00', 70),('01.01.2014 17:45:00', 71),
                 ('01.01.2014 18:00:00', 72),('01.01.2014 18:15:00', 73),('01.01.2014 18:30:00', 74) , 
                 ('01.01.2014 18:45:00', 75) , ('01.01.2014 19:00:00', 76) , ('01.01.2014 19:15:00', 77),
                 ('01.01.2014 19:30:00', 78),('01.01.2014 19:45:00', 79),('01.01.2014 20:00:00', 80),
                 ('01.01.2014 20:15:00', 81),('01.01.2014 20:30:00', 82),('01.01.2014 20:45:00', 83),
                 ('01.01.2014 21:00:00',84),('01.01.2014 21:15:00', 85) , ('01.01.2014 21:30:00', 86),
                 ('01.01.2014 21:45:00', 87),('01.01.2014 22:00:00', 88),('01.01.2014 22:15:00', 89),
                 ('01.01.2014 22:30:00', 90),('01.01.2014 22:45:00', 91),('01.01.2014 23:00:00', 92),
                 ('01.01.2014 23:15:00', 93),('01.01.2014 23:30:00', 94) , ('01.01.2014 23:45:00', 95)])
def ev1arrdeptime():
    with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv') as f:
        reader = csv.reader(f)
        header = next(reader)
        y = []
        for row in reader :
            ydist = float(row[0])
            yaDate = row[1]
            ydDate = row[2]
            y.append([ydist,yaDate,ydDate])
        ev1 =(random.choice(y))
    return(ev1);
def parameterdy( i, T, IE,RE,TS,TD):
    M= i
    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/household8_profile.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHP_a = float(row[0])
        HHP_b = float(row[1])
        HHP_c = float(row[2])
        HHQ_a = float(row[3])
        HHQ_b = float(row[4])
        HHQ_c = float(row[5])
        HH.append([HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PV_P = float(row[0])
        PV.append(PV_P)
    m = Model("dynamicscheduling")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    #Phase constraint
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
            
    #SOC user constraint
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            e[i][t] +=  p[i,t] * eff * 1/4  # Energy consumed by EV1   1/4 *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]
            te[i] += e[i][t]
            n = te[i]
        initial = IE[i] 
        required = RE 
        I_E = int(np.round(initial))
        R_E = int(np.round(required))
        m.addConstr(n >=0,name="SOC_minconstraint[%d]" %i)
        m.addConstr(n <=24,name="SOC_maxconstraint[%d]" %i)
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
    
    #power outside TS TD as 0
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))

    #Grid constraint
    for t in range(STime[i],DTime[i]):
        for i in range(M):
    #if i < M :
            pa[i][t] = a[i,t] * p[i,t]
            pb[i][t] = b[i,t] * p[i,t]
            pc[i][t] = c[i,t] * p[i,t]
            
            patot[t] += pa[i][t]
            pbtot[t] += pb[i][t]
            pctot[t] += pc[i][t]
        #print(pa[i][t])
        pahhpv[t] = patot[t] + HH[t][0] - PV[t]
        pbhhpv[t] = pbtot[t] + HH[t][1]
        pchhpv[t] = pctot[t] + HH[t][2]
    
        pab[t] = pahhpv[t] - pbhhpv[t]
        pbc[t] = pbhhpv[t] - pchhpv[t]
        pca[t] = pchhpv[t] - pahhpv[t]
    
        m.addConstr(pab[t] <= 4.60, name="Power_A_B_maxconstraint")
        m.addConstr(pab[t] >= -4.60, name="Power_A_B_minconstraint")
        m.addConstr(pbc[t] <= 4.60, name="Power_B_C_maxconstraint")
        m.addConstr(pbc[t] >= -4.60, name="Power_B_C_minconstraint")
        m.addConstr(pca[t] <= 4.60, name="Power_C_A_maxconstraint")
        m.addConstr(pca[t] >= -4.60, name="Power_C_A_minconstraint")
    
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    Im_cost = C
    ST = STime[0]
    i = M - 1
    DT = DTime[i]
    for t in range(ST,DT):
        if pab[t] <= 2.3:
            Im_cost += 0.0
        elif pab[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pbc[t] <= 2.3:
            Im_cost += 0.0
        elif pbc[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pca[t] <= 2.3:
            Im_cost += 0.0
        elif pca[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
    #set objective:
    m.ModelSense = GRB.MINIMIZE
    Im_C = Im_cost
    #m.setObjective(C)
    m.setObjective(Im_C)

    m.write('testDynamicscheduling.lp')
    #m.write('testcost.lp')

    # Compute optimal solution
    m.optimize()
    m.write('testDynamicscheduling.sol')
    #print solution 

    for v in m.getVars():
        print(v.varName, v.x)
        
    print('Obj:', m.objVal)
    finalcost = m.objVal
         
       # elif i ==1:
       #     time.sleep(30)
       #     Ev2 = ev2adt() #ev2arrdeptime()
       #     print('ev2:',Ev2)
       #     print('ev2_a:',Ev2[1])
       #     print('ev2_d:',Ev2[2])
    return(finalcost);
M = 2
T = 96
#power=[0 for i in range(T)]
InitialEgy = [0 for i in range(M)]
energycost1 = [0 for i in range(M)]
IEgy = [0 for i in range(M)] 
ev_a=[0 for i in range(M)]
ev_d=[0 for i in range(M)]
Ev = [0 for i in range(M)]
IE = [0 for i in range(M)]
RE = 17.0
Ev[0] = ev1arrdeptime()
Ev[1] = ev1arrdeptime()
Date = '01.01.2014 '
Time = ':00'
Starttime =[0 for i in range(M)]
Departtime =[0 for i in range(M)]
for i in range(0,M):
    Starttime[i] = Date + Ev[i][1] +Time
    Departtime[i] = Date + Ev[i][2] +Time
    print('energy:',Ev[i][0])
    IE[i] = RE - Ev[i][0]
    print('Initial energy:',IE[i])
    print('ev_a:',Starttime[i])
    print('ev_d:',Departtime[i])
    for item,val in thisdict.items():
        if item == Starttime[i]:
            ev_a[i] = val
        elif item == Departtime[i]:
            ev_d[i] = val
if ev_a[0] < ev_a[1]:        
    TS = [ev_a[0],ev_a[1]]  
    TD= [ev_d[0],ev_d[1]] 
    print('ev1 arrives before')
    print(TS)
    print(TD)
else:
    TS = [ev_a[1],ev_a[0]]   
    TD= [ev_d[1],ev_d[0]]
    print('ev2 arrives before')
    print(TS)
    print(TD)
if TD[0] > TS[1] and TD[0] < 96 and TD[1] > TS[1] and TD[1] < 96:
    if TD[0] < TD[1] or TD[1] < TD[0] or TD[1] == TD[0]:
        print('case 1: Both EVs depart before mid-night' )
        Start = [TS[0]]
        Depart = [TD[0]]
        cost1=parameterdy(1,96,IE,RE,Start,Depart)

        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)            
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
            
        with open('testDynamicscheduling1ev.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            power =[]
            p = []
            for row in csv_reader:
                power_str = row[0]
                power_P = float(row[1])
                power.append([power_str,power_P])
            for i in range(len(power)):
                for j in range(TS[0],TS[1]):
                    if power[i][0] == 'Power[%d,%d]' %(0,j):
                        p.append(power[i][1])
            print(p)
            energy =[]
            pwr=[0 for i in range(T)]
            for i in range(len(p)):
                pwr[i] = p[i] *eff * 1/4
                energy.append(pwr[i])
                InitialEgy[0] += energy[i]
            print(energy)
            print(InitialEgy[0])
            path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
            file2 = open(path2, newline='')
            reader2 = csv.reader(file2)
            header2 = next(reader2)
            marketcost = []
            for row in reader2 :
                mcost = float(row[0])
                marketcost.append(mcost)
            cost = [j/100 for j in marketcost]
            i = 0
            for t in range(TS[0],TS[1]):
                energycost1[0] += energy[i] * cost[t]
                i +=1
            print(energycost1[0])            
        Start = [TS[1],TS[1]]
        Depart = [TD[0],TD[1]]
        IE[0] += InitialEgy[0]
        cost2=parameterdy(2,96,IE,RE,Start,Depart)
        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
    
        f.close()
        myfile.close()
        finalcost = energycost1[0] + cost2
        print('final optimal cost:',finalcost)
    
elif TD[0] < TS[0] and TD[0] >= 0 and TD[1] < TS[0] and TD[1] >= 0:
    if TD[0] < TD[1] or TD[1] < TD[0]:
        print('case2: Both EVs depart in the morning')
        Start = [TS[0]]
        Depart = [96]
        cost1=parameterdy(1,96,IE,RE,Start,Depart)

        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)            
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
            
        with open('testDynamicscheduling1ev.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            power =[]
            p = []
            for row in csv_reader:
                power_str = row[0]
                power_P = float(row[1])
                power.append([power_str,power_P])
            for i in range(len(power)):
                for j in range(TS[0],TS[1]):
                    if power[i][0] == 'Power[%d,%d]' %(0,j):
                        p.append(power[i][1])
            print(p)
            energy =[]
            pwr=[0 for i in range(T)]
            for i in range(len(p)):
                pwr[i] = p[i] *eff * 1/4
                energy.append(pwr[i])
                InitialEgy[0] += energy[i]
            print(energy)
            print(InitialEgy[0])
            path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
            file2 = open(path2, newline='')
            reader2 = csv.reader(file2)
            header2 = next(reader2)
            marketcost = []
            for row in reader2 :
                mcost = float(row[0])
                marketcost.append(mcost)
            cost = [j/100 for j in marketcost]    
            i = 0
            for t in range(TS[0],TS[1]):
                energycost1[0] += energy[i] * cost[t]
                i += 1
            print(energycost1[0])            
        Start = [TS[1],TS[1]]
        Depart = [96,96]
        IE[0] += InitialEgy[0]
        cost2=parameterdy(2,96,IE,RE,Start,Depart)
        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
        with open('testDynamicscheduling2evs.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            powr =[]
            po = []
            for row in csv_reader:
                powr_str = row[0]
                powr_P = float(row[1])
                powr.append([powr_str,powr_P])
            for i in range(len(powr)):
                for j in range(TS[1],96):
                    if powr[i][0] == 'Power[%d,%d]' %(0,j) or powr[i][0] == 'Power[%d,%d]' %(1,j):
                        po.append(powr[i][1])
            print(po)
            energy =[]
            pwrs=[0 for i in range(T)]
            for i in range(len(po)):
                pwrs[i] = po[i] *eff * 1/4
                energy.append(pwrs[i])
                IEgy[0] += energy[i]
            print(energy)
            print(IEgy[0])           
        Start = [0,0]
        Depart = [TD[0],TD[1]]
        IE[0] += IEgy[0]
        cost3=parameterdy(2,96,IE,RE,Start,Depart) 
        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling2evsfinal.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
    
        f.close()
        myfile.close()
        finalcost = energycost1[0] + cost2 + cost3
        print('final optimal cost:',finalcost)
                
    else:
        exit

energy: 7.31
Initial energy: 9.690000000000001
ev_a: 01.01.2014 18:00:00
ev_d: 01.01.2014 10:30:00
energy: 7.99
Initial energy: 9.01
ev_a: 01.01.2014 17:15:00
ev_d: 01.01.2014 08:15:00
ev2 arrives before
[69, 72]
[33, 42]
case2
Optimize a model with 168 rows, 384 columns and 438 nonzeros
Model has 162 quadratic constraints
Variable types: 96 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [7e-02, 8e-02]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [3e-01, 9e+00]
Presolve removed 140 rows and 276 columns
Presolve time: 0.04s
Presolved: 676 rows, 432 columns, 1890 nonzeros
Variable types: 351 continuous, 81 integer (81 binary)

Root relaxation: objective 1.897607e+00, 258 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

  

  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [3e-01, 9e+00]
Presolve removed 292 rows and 576 columns
Presolve time: 0.05s
Presolved: 1058 rows, 768 columns, 3360 nonzeros
Variable types: 624 continuous, 144 integer (144 binary)

Root relaxation: objective 3.204552e+00, 441 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.20455    0   60          -    3.20455      -     -    0s
H    0     0                       3.5577671    3.20455  9.93%     -    0s
H    0     0                       3.5402662    3.20455  9.48%     -    0s
H    0     0                       3.4265914    3.20455  6.48%     -    0s
     0     0    3.20455    0   76    3.42659    3.20455  6.48%     -    0s
H    0     0                       3.3851517    3.20455  5.34%     -    0s
     0     0    3.20455    0   76    3.38515    3.20455 

 669210 402652    3.33660   48   66    3.36457    3.33261  0.95%  14.1  315s
 679569 409081    3.35964   51   56    3.36457    3.33266  0.95%  14.1  320s
 689907 415421    3.36213   72   24    3.36457    3.33270  0.95%  14.1  325s
 700157 421821    3.34538   53   57    3.36457    3.33274  0.95%  14.1  330s
 710505 428157     cutoff   70         3.36457    3.33278  0.94%  14.1  335s
 720132 434147    3.34752   58   48    3.36457    3.33281  0.94%  14.1  340s
 730750 440746    3.36329   72   30    3.36457    3.33286  0.94%  14.1  345s
 740608 446758    3.34513   55   57    3.36457    3.33289  0.94%  14.1  350s
 750829 452972    3.36326   62   36    3.36457    3.33293  0.94%  14.0  355s
 761004 459021    3.33889   46   60    3.36457    3.33298  0.94%  14.0  360s
 770876 464917    3.35006   50   63    3.36457    3.33303  0.94%  14.0  365s
 781035 471222    3.36452   83   10    3.36457    3.33309  0.94%  14.0  371s
 789444 476334    3.33853   49   68    3.36457    3.33312  0.93%  14.0  375s

 1525802 900282    3.36303   74   23    3.36457    3.33527  0.87%  14.1  846s
 1531911 903652    3.35924   69   31    3.36457    3.33528  0.87%  14.1  851s
 1538092 907086    3.34862   50   65    3.36457    3.33530  0.87%  14.1  855s
 1542794 909687    3.36061   62   43    3.36457    3.33531  0.87%  14.1  860s
 1551423 914361    3.35735   55   48    3.36457    3.33532  0.87%  14.1  866s
 1557632 917786    3.34341   51   66    3.36457    3.33534  0.87%  14.1  870s
 1563652 921097    3.34005   45   77    3.36457    3.33535  0.87%  14.1  875s
 1571549 925405    3.34680   56   53    3.36457    3.33536  0.87%  14.1  881s
 1577987 928864    3.36236   69   25    3.36457    3.33537  0.87%  14.1  885s
 1583962 932136    3.33990   48   68    3.36457    3.33539  0.87%  14.1  890s
 1592303 936735    3.36450   69   26    3.36457    3.33541  0.87%  14.2  896s
 1598523 940186    3.36317   70   32    3.36457    3.33542  0.87%  14.2  900s
 1606386 944442     cutoff   68         3.36457    3.33544  0.87

 2161137 1251302    3.35388   62   42    3.36457    3.33636  0.84%  14.3 1370s
 2167295 1254620    3.35452   59   48    3.36457    3.33637  0.84%  14.3 1375s
 2173553 1258094    3.36378   64   37    3.36457    3.33637  0.84%  14.3 1381s
 2179472 1261416     cutoff   59         3.36457    3.33638  0.84%  14.3 1386s
 2183641 1263624    3.36411   70   27    3.36457    3.33638  0.84%  14.3 1390s
 2189412 1266874     cutoff   73         3.36457    3.33639  0.84%  14.3 1395s
 2195641 1270262    3.34778   52   51    3.36457    3.33640  0.84%  14.3 1400s
 2201807 1273740    3.36209   68   36    3.36457    3.33641  0.84%  14.3 1406s
 2208142 1277110    3.34021   44   69    3.36457    3.33642  0.84%  14.3 1411s
 2212130 1279244    3.35550   65   42    3.36457    3.33642  0.84%  14.3 1415s
 2218287 1282687     cutoff   67         3.36457    3.33643  0.84%  14.3 1420s
 2224609 1286177    3.36425   71   29    3.36457    3.33644  0.84%  14.3 1425s
 2230583 1289570    3.34678   54   59    3.36457    

...

 7996869 4308622     cutoff   66         3.36457    3.33996  0.73%  14.6 13954s
 7999023 4309714    3.35843   53   58    3.36457    3.33996  0.73%  14.6 13962s
 8001148 4310790    3.35367   62   46    3.36457    3.33996  0.73%  14.6 13970s
 8003361 4311876    3.36307   66   30    3.36457    3.33996  0.73%  14.6 13978s
 8005500 4313131    3.35819   56   51    3.36457    3.33996  0.73%  14.6 13985s
 8007035 4313801    3.34331   53   61    3.36457    3.33996  0.73%  14.6 13993s
 8009044 4314873     cutoff   63         3.36457    3.33996  0.73%  14.6 14000s
 8010981 4315869    3.34470   47   67    3.36457    3.33996  0.73%  14.6 14008s
 8013103 4316928    3.34892   52   59    3.36457    3.33997  0.73%  14.6 14016s
 8015137 4317966     cutoff   61         3.36457    3.33997  0.73%  14.6 14024s
 8017417 4319167     cutoff   55         3.36457    3.33997  0.73%  14.6 14032s
 8019277 4320122     cutoff   57         3.36457    3.33997  0.73%  14.6 14041s
 8021502 4321286    3.36439   69   25   

 8204398 4413445    3.35944   62   38    3.36457    3.34003  0.73%  14.6 14780s
 8206432 4414495    3.34458   54   59    3.36457    3.34003  0.73%  14.6 14789s
 8208657 4415635    3.35268   57   50    3.36457    3.34003  0.73%  14.6 14797s
 8210662 4416650     cutoff   46         3.36457    3.34003  0.73%  14.6 14805s
 8212857 4417744    3.35092   54   50    3.36457    3.34003  0.73%  14.6 14813s
 8214898 4418773    3.34526   51   53    3.36457    3.34003  0.73%  14.6 14821s
 8216903 4419762    3.34450   55   54    3.36457    3.34003  0.73%  14.6 14830s
 8219122 4420922    3.34926   51   54    3.36457    3.34003  0.73%  14.6 14838s
 8221263 4421898    3.36444   61   41    3.36457    3.34003  0.73%  14.6 14846s
 8223372 4422980    3.35839   55   56    3.36457    3.34003  0.73%  14.6 14855s
 8224906 4423704    3.35659   63   45    3.36457    3.34003  0.73%  14.6 14863s
 8227025 4424774     cutoff   79         3.36457    3.34003  0.73%  14.6 14871s
 8229296 4425973     cutoff   68        

 8414694 4519627    3.36062   64   44    3.36457    3.34009  0.73%  14.6 15616s
 8416478 4520557    3.36079   63   29    3.36457    3.34009  0.73%  14.6 15624s
 8418676 4521659    3.36209   64   38    3.36457    3.34009  0.73%  14.6 15632s
 8420796 4522798    3.35203   58   42    3.36457    3.34010  0.73%  14.6 15641s
 8422894 4523815    3.36429   62   45    3.36457    3.34010  0.73%  14.6 15649s
 8424963 4524874    3.35902   63   43    3.36457    3.34010  0.73%  14.6 15657s
 8427204 4526001    3.35174   58   50    3.36457    3.34010  0.73%  14.6 15665s
 8429201 4527011     cutoff   61         3.36457    3.34010  0.73%  14.6 15673s
 8431365 4528128    3.36075   66   38    3.36457    3.34010  0.73%  14.6 15681s
 8433346 4529139    3.34960   59   50    3.36457    3.34010  0.73%  14.6 15689s
 8435441 4530222    3.36410   69   19    3.36457    3.34010  0.73%  14.6 15697s
 8437260 4531092    3.36374   64   35    3.36457    3.34010  0.73%  14.6 15705s
 8439297 4532166    3.36382   67   35   

 8625111 4626477    3.34971   54   60    3.36457    3.34016  0.73%  14.6 16490s
 8627195 4627486    3.35435   60   44    3.36457    3.34016  0.73%  14.6 16498s
 8629306 4628555    3.36269   64   38    3.36457    3.34016  0.73%  14.6 16506s
 8631136 4629575    3.35924   66   33    3.36457    3.34017  0.73%  14.6 16514s
 8633272 4630658    3.35412   53   50    3.36457    3.34017  0.73%  14.6 16523s
 8635376 4631729    3.34961   55   54    3.36457    3.34017  0.73%  14.6 16531s
 8637456 4632814    3.36257   57   37    3.36457    3.34017  0.73%  14.6 16539s
 8639421 4633853    3.36181   59   45    3.36457    3.34017  0.73%  14.6 16546s
 8641110 4634644    3.35347   54   51    3.36457    3.34017  0.73%  14.6 16554s
 8643267 4635739    3.34282   54   59    3.36457    3.34017  0.73%  14.6 16563s
 8645218 4636676    3.35374   62   43    3.36457    3.34017  0.73%  14.6 16571s
 8647330 4637779     cutoff   60         3.36457    3.34017  0.73%  14.6 16579s
 8649404 4638862    3.35469   61   41   

 8834170 4733826    3.36375   65   33    3.36457    3.34023  0.72%  14.6 17355s
 8836056 4734743     cutoff   55         3.36457    3.34023  0.72%  14.6 17363s
 8838150 4735791    3.36455   62   28    3.36457    3.34023  0.72%  14.6 17372s
 8840260 4736833    3.35648   62   48    3.36457    3.34024  0.72%  14.6 17381s
 8842094 4737736    3.35974   63   48    3.36457    3.34024  0.72%  14.6 17389s
 8844272 4738850     cutoff   62         3.36457    3.34024  0.72%  14.6 17399s
 8846296 4739891    3.36277   52   62    3.36457    3.34024  0.72%  14.6 17408s
 8848463 4740959    3.34801   55   54    3.36457    3.34024  0.72%  14.6 17416s
 8850537 4742026    3.36244   66   38    3.36457    3.34024  0.72%  14.6 17426s
 8852424 4742976    3.34519   51   55    3.36457    3.34024  0.72%  14.6 17434s
 8854525 4744047    3.35719   59   44    3.36457    3.34024  0.72%  14.6 17443s
 8856610 4745133     cutoff   61         3.36457    3.34024  0.72%  14.6 17451s
 8858776 4746219    3.36298   63   39   

In [ ]:
if TD[0] > TS[1] and TD[0] < 96 and TD[1] > TS[1] and TD[1] < 96:
    if TD[0] < TD[1] or TD[1] < TD[0] or TD[1] == TD[0]:
        print('case 1: Both EVs depart before mid-night')
        Start = [TS[0]]
        Depart = [TD[0]]
        cost1=parameterdy(1,96,IE,RE,Start,Depart)

        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)            
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
            
        with open('testDynamicscheduling1ev.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            power =[]
            p = []
            for row in csv_reader:
                power_str = row[0]
                power_P = float(row[1])
                power.append([power_str,power_P])
            for i in range(len(power)):
                for j in range(TS[0],TS[1]):
                    if power[i][0] == 'Power[%d,%d]' %(0,j):
                        p.append(power[i][1])
            print(p)
            energy =[]
            pwr=[0 for i in range(T)]
            for i in range(len(p)):
                pwr[i] = p[i] *eff * 1/4
                energy.append(pwr[i])
                InitialEgy[0] += energy[i]
            print(energy)
            print(InitialEgy[0])
            path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
            file2 = open(path2, newline='')
            reader2 = csv.reader(file2)
            header2 = next(reader2)
            marketcost = []
            for row in reader2 :
                mcost = float(row[0])
                marketcost.append(mcost)
            cost = [j/100 for j in marketcost]
            i = 0
            for t in range(TS[0],TS[1]):
                energycost1[0] += energy[i] * cost[t]
                i +=1
            print(energycost1[0])            
        Start = [TS[1],TS[1]]
        Depart = [TD[0],TD[1]]
        IE[0] += InitialEgy[0]
        cost2=parameterdy(2,96,IE,RE,Start,Depart)
        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
    
        f.close()
        myfile.close()
        finalcost = energycost1[0] + cost2
        print('final optimal cost:',finalcost)
    
elif TD[0] < TS[0] and TD[0] >= 0 and TD[1] < TS[0] and TD[1] >= 0:
    if TD[0] < TD[1] or TD[1] < TD[0]:
        print('case2: Both EVs depart in the morning')
        Start = [TS[0]]
        Depart = [96]
        cost1=parameterdy(1,96,IE,RE,Start,Depart)

        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)            
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
            
        with open('testDynamicscheduling1ev.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            power =[]
            p = []
            for row in csv_reader:
                power_str = row[0]
                power_P = float(row[1])
                power.append([power_str,power_P])
            for i in range(len(power)):
                for j in range(TS[0],TS[1]):
                    if power[i][0] == 'Power[%d,%d]' %(0,j):
                        p.append(power[i][1])
            print(p)
            energy =[]
            pwr=[0 for i in range(T)]
            for i in range(len(p)):
                pwr[i] = p[i] *eff * 1/4
                energy.append(pwr[i])
                InitialEgy[0] += energy[i]
            print(energy)
            print(InitialEgy[0])
            path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
            file2 = open(path2, newline='')
            reader2 = csv.reader(file2)
            header2 = next(reader2)
            marketcost = []
            for row in reader2 :
                mcost = float(row[0])
                marketcost.append(mcost)
            cost = [j/100 for j in marketcost]    
            i = 0
            for t in range(TS[0],TS[1]):
                energycost1[0] += energy[i] * cost[t]
                i += 1
            print(energycost1[0])            
        Start = [TS[1],TS[1]]
        Depart = [96,96]
        IE[0] += InitialEgy[0]
        cost2=parameterdy(2,96,IE,RE,Start,Depart)
        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
        with open('testDynamicscheduling2evs.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            powr =[]
            po = []
            for row in csv_reader:
                powr_str = row[0]
                powr_P = float(row[1])
                powr.append([powr_str,powr_P])
            for i in range(len(powr)):
                for j in range(TS[1],96):
                    if powr[i][0] == 'Power[%d,%d]' %(0,j) or powr[i][0] == 'Power[%d,%d]' %(1,j):
                        po.append(powr[i][1])
            print(po)
            energy =[]
            pwrs=[0 for i in range(T)]
            for i in range(len(po)):
                pwrs[i] = po[i] *eff * 1/4
                energy.append(pwrs[i])
                IEgy[0] += energy[i]
            print(energy)
            print(IEgy[0])           
        Start = [0,0]
        Depart = [TD[0],TD[1]]
        IE[0] += IEgy[0]
        cost3=parameterdy(2,96,IE,RE,Start,Depart) 
        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling2evsfinal.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
    
        f.close()
        myfile.close()
        finalcost = energycost1[0] + cost2 + cost3
        print('final optimal cost:',finalcost)
elif TD[0] > TS[1] and TD[0] < 96 and TD[1] < TS[0] and TD[1] >= 0:
    print('case3: EV1 depart before mid-night and EV2 departs in the morning')
    Start = [TS[0]]
    Depart = [TD[0]]
    cost1=parameterdy(1,96,IE,RE,Start,Depart)

    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)            
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
            
    with open('testDynamicscheduling1ev.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        power =[]
        p = []
        for row in csv_reader:
            power_str = row[0]
            power_P = float(row[1])
            power.append([power_str,power_P])
        for i in range(len(power)):
            for j in range(TS[0],TS[1]):
                if power[i][0] == 'Power[%d,%d]' %(0,j):
                    p.append(power[i][1])
        print(p)
        energy =[]
        pwr=[0 for i in range(T)]
        for i in range(len(p)):
            pwr[i] = p[i] *eff * 1/4
            energy.append(pwr[i])
            InitialEgy[0] += energy[i]
        print(energy)
        print(InitialEgy[0])
        path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
        file2 = open(path2, newline='')
        reader2 = csv.reader(file2)
        header2 = next(reader2)
        marketcost = []
        for row in reader2 :
            mcost = float(row[0])
            marketcost.append(mcost)
        cost = [j/100 for j in marketcost]    
        i = 0
        for t in range(TS[0],TS[1]):
            energycost1[0] += energy[i] * cost[t]
            i += 1
        print(energycost1[0])            
    Start = [TS[1],TS[1]]
    Depart = [TD[0],96]
    IE[0] += InitialEgy[0]
    cost2=parameterdy(2,96,IE,RE,Start,Depart)
    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    with open('testDynamicscheduling2evs.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        powr =[]
        po = []
        for row in csv_reader:
            powr_str = row[0]
            powr_P = float(row[1])
            powr.append([powr_str,powr_P])
        for i in range(len(powr)):
            for j in range(TS[1],96):
                if powr[i][0] == 'Power[%d,%d]' %(1,j):
                    po.append(powr[i][1])
        print(po)
        energy =[]
        pwrs=[0 for i in range(T)]
        for i in range(len(po)):
            pwrs[i] = po[i] *eff * 1/4
            energy.append(pwrs[i])
            IEgy[0] += energy[i]
        print(energy)
        print(IEgy[0])           
    Start = [0]
    Depart = [TD[1]]
    IE[0] += IEgy[0]
    cost3=parameterdy(1,96,IE,RE,Start,Depart) 
    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling2evsfinal.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
    finalcost = energycost1[0] + cost2 + cost3
    print('final optimal cost:',finalcost)       
elif TD[1] > TS[1] and TD[1] < 96 and TD[0] < TS[0] and TD[0] >= 0:
    print('case4: EV2 depart before mid-night and EV1 departs in the morning')
    Start = [TS[0]]
    Depart = [96]
    cost1=parameterdy(1,96,IE,RE,Start,Depart)

    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)            
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
            
    with open('testDynamicscheduling1ev.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        power =[]
        p = []
        for row in csv_reader:
            power_str = row[0]
            power_P = float(row[1])
            power.append([power_str,power_P])
        for i in range(len(power)):
            for j in range(TS[0],TS[1]):
                if power[i][0] == 'Power[%d,%d]' %(0,j):
                    p.append(power[i][1])
        print(p)
        energy =[]
        pwr=[0 for i in range(T)]
        for i in range(len(p)):
            pwr[i] = p[i] *eff * 1/4
            energy.append(pwr[i])
            InitialEgy[0] += energy[i]
        print(energy)
        print(InitialEgy[0])
        path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
        file2 = open(path2, newline='')
        reader2 = csv.reader(file2)
        header2 = next(reader2)
        marketcost = []
        for row in reader2 :
            mcost = float(row[0])
            marketcost.append(mcost)
        cost = [j/100 for j in marketcost]    
        i = 0
        for t in range(TS[0],TS[1]):
            energycost1[0] += energy[i] * cost[t]
            i += 1
        print(energycost1[0])            
    Start = [TS[1],TS[1]]
    Depart = [96,TD[1]]
    IE[0] += InitialEgy[0]
    cost2=parameterdy(2,96,IE,RE,Start,Depart)
    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    with open('testDynamicscheduling2evs.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        powr =[]
        po = []
        for row in csv_reader:
            powr_str = row[0]
            powr_P = float(row[1])
            powr.append([powr_str,powr_P])
        for i in range(len(powr)):
            for j in range(TS[1],96):
                if powr[i][0] == 'Power[%d,%d]' %(0,j):
                    po.append(powr[i][1])
        print(po)
        energy =[]
        pwrs=[0 for i in range(T)]
        for i in range(len(po)):
            pwrs[i] = po[i] *eff * 1/4
            energy.append(pwrs[i])
            IEgy[0] += energy[i]
        print(energy)
        print(IEgy[0])           
    Start = [0]
    Depart = [TD[0]]
    IE[0] += IEgy[0]
    cost3=parameterdy(1,96,IE,RE,Start,Depart) 
    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling2evsfinal.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
    finalcost = energycost1[0] + cost2 + cost3
    print('final optimal cost:',finalcost)  
else:
    exit